# Kiểm tra readme trước

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
cd /content/gdrive/MyDrive/PTTKGT/data

/content/gdrive/MyDrive/PTTKGT/data


# Thuật toán tham khảo - TUHUFP

## Thêm các thư viện tính toán thời gian và bộ nhớ

In [3]:
import sys, os
import time
import tracemalloc

## Cài đặt cấu trúc dữ liệu

### Cài đặt Tep

In [4]:
class Tep:
    """
    Đối tượng lưu trữ các thông tin của i ứng với giao dịch (TID, xác xuất, tiện ích, tiện ích giao dịch)
    """
    def __init__(self, TID: int, prob: float, util: int, trans_util: int):
        """
        Khởi tạo tep

        Args:
            TID (int): mã giao dịch
            prob (float): xác xuất
            util (int): giá trị tiện ích
            trans_util (int): tiện ích giao dịch
        """
        self.TID = TID
        self.prob = round(prob, 2)
        self.util = util
        self.trans_util = trans_util

    def combine_with(self, other_tep):
        """
        Kết hợp 2 tep với nhau

        Args:
            other_tep (Tep): Tep cần kết hợp

        Returns:
            Tep: một tep mới
        """
        combined_prob = round(self.prob * other_tep.prob, 2)
        combined_util = self.util + other_tep.util
        return Tep(self.TID, combined_prob, combined_util, other_tep.trans_util)

    def __repr__(self):
        return f"Tep(TID={self.TID}, prob={self.prob}, util={self.util}, trans_util={self.trans_util})\n"

### Cài đặt Cup

In [5]:
class Cup:
    """
    Đối tượng lưu trữ các thông tin của một i bao gồm: tên, độ hỗ trợ mong đợi, danh sách Tep,
        giá trị xác xuất lớn nhất trong Tep, TWU, giá trị tiện ích, phần tử cuối của i
    """
    def __init__(self, name, tep_list=None):
        """
        Khởi tạo Cup

        Args:
            name (str): tên của cup hay tên của i
            tep_list (List[Tep], None): danh sách Tep của i
        """
        self.name = name
        self.exp_sup = round(sum(tep.prob for tep in tep_list), 2) if tep_list else 0
        self.tep_list = tep_list if tep_list else []
        self.max_prob = round(max(tep.prob for tep in tep_list), 2) if tep_list else 0
        self.trans_wei_util = sum(tep.trans_util for tep in tep_list) if tep_list else 0
        self.utility = sum(tep.util for tep in tep_list) if tep_list else 0
        self.last = []

    def update(self, probability: float, TID, trans_util, util_value) -> None:
        """
        Cập nhật lại các giá trị trong Cup

        Args:
            probability (float): xác xuất tồn tại
            TID (_type_): mã giao dịch
            trans_util (_type_): tiện ích giao dịch
            util_value (_type_): giá trị tiện ích
        """
        probability = round(probability, 2)
        tep = Tep(TID, probability, util_value, trans_util)
        self.exp_sup = round(self.exp_sup + probability, 2)
        self.utility += util_value
        self.tep_list.append(tep)
        self.max_prob = max(self.max_prob, probability)
        self.trans_wei_util += trans_util

    def combine_tep(self, tep_list_x, tep_list_y):
        """
        Kết hợp hai danh sách Tep

        Args:
            tep_list_x (List[Tep]): danh sách Tep của i X
            tep_list_y (List[Tep]): danh sách Tep của i Y

        Returns:
            List[Tep]: danh sách Tep của i XY
        """
        tep_list_xy = []
        i, j = 0, 0
        while i < len(tep_list_x) and j < len(tep_list_y):
            tX = tep_list_x[i]
            tY = tep_list_y[j]
            if tX.TID < tY.TID:
                i += 1
            elif tX.TID > tY.TID:
                j += 1
            else:
                combined_prob = round(tX.prob * tY.prob, 2)
                combined_util = tX.util + tY.util
                tep_list_xy.append(Tep(tX.TID, combined_prob, combined_util, tX.trans_util))
                i += 1
                j += 1
        return tep_list_xy

    def combine_with(self, other_cup):
        """
        Kết hợp hai Cup lại với nhau

        Args:
            other_cup (Cup): Cup tham gia kết hợp

        Returns:
            Cup: một Cup mới
        """
        if len(other_cup.last) == 0:
            combined_name = self.name + ', ' + other_cup.name
            combined_tep_list = self.combine_tep(self.tep_list, other_cup.tep_list)
            last = [other_cup]
        else:
            combined_name = self.name + ', ' + other_cup.last[0].name
            combined_tep_list = self.combine_tep(self.tep_list, other_cup.last[0].tep_list)
            last = other_cup.last

        combined_cup = Cup(combined_name, combined_tep_list)
        combined_cup.last = last
        return combined_cup

    def __repr__(self) -> str:
        return f"Cup(name={self.name}, exp_sup={self.exp_sup}, utility={self.utility})"

## Cài đặt thuật toán

In [6]:
class AlgorithmTUHUFP:
    """
    Triển khai thuật toán khai thác top-k mẫu phổ biến tiện ích cao không chắc chắn
    """
    def __init__(self):
        self.start_timestamp = 0
        self.end_timestamp = 0
        self.database_size = 0
        self.database_util = 0
        self.candidates = 0
        self.top_UHUFP = []
        self.single_cup = {}
        self.threshold = float('-inf')
        self.min_util = float('-inf')
        self.peak_memory_usage = 0

    def read_data(self, file_path: str, percentage: float, k: int) -> None:
        """
        Đọc các giá trị từ cơ sở dũ liệu

        Args:
            file_path (str): đường dẫn chứa file cơ sở dữ liệu
            percentage (float): ngưỡng tiện ích (%)
            k (int): số lượng mẫu cần tìm
        """
        file_paths = file_path.split(", ")
        # print(f"File paths: {file_paths}")  \
        try:
            with open(file_paths[0], 'r') as file1, open(file_paths[1], 'r') as file2:
                print("Reading data . . .")
                tlines = file1.readlines()
                ulines = file2.readlines()
                i_name = tlines[0].strip().split(" ")
                # print(f"i names: {i_name}")

                TID = 1
                for prob_line, util_line in zip(tlines[1:], ulines):
                    # print(f"Processing TID {TID}: {prob_line.strip()} | {util_line.strip()}")  # Check each line
                    self.process_data(i_name, prob_line.strip(), util_line.strip(), TID)
                    TID += 1
                    self.database_size += 1

                self.min_util = int(self.database_util * percentage)
                print(f"Minimum utility set to: {self.min_util}")
        except FileNotFoundError as e:
            print(f"File not found: {e}")
            print("STOP ALGORITHM !!!")
            sys.exit(0)
        except Exception as e:
            print(f"An error occurred: {e}")
            sys.exit(0)


    def process_data(self, i_name, prob_line: float, util_line: int, TID: int) -> None:
        """
        Xử lí dữ liệu và tạo Cup

        Args:
            i_name (List[str]): danh sách các i
            prob_line (float): danh sách các xác xuất của i
            util_line (int): danh sách tiện ích của i
            TID (int): mã giao dịch
        """
        prob_list = prob_line.split(" ")
        trans = util_line.split(":")
        is_util = trans[0].split(" ")
        # print(is_util)
        trans_util = int(trans[1])
        util_list = trans[2].split(" ")

        i_util_list = {i: int(util) for i, util in zip(is_util, util_list)}


        self.database_util += trans_util

        for i, prob in enumerate(prob_list):
            prob = prob.strip()
            if prob and float(prob) > 0:
                try:
                    i = i_name[i]
                    probability = round(float(prob), 2)  # Round to 2 decimal places

                    util_value = i_util_list[i]
                    cup_name = i
                    #kiểm tra cup đã được tạo hay chưa
                    if cup_name in self.single_cup:
                        self.single_cup[cup_name].update(probability, TID, trans_util, util_value)
                    else:
                        new_tep = Tep(TID, probability, util_value, trans_util)
                        self.single_cup[cup_name] = Cup(cup_name, [new_tep])
                except ValueError as ve:
                    print(f"Error processing probability '{prob}': {ve}")
                    continue

    def combine_cup(self, cupX, cupY):
        """
        Kết hơp hai cup X và Y

        Args:
            cupX (Cup): Cup tham gia kết hợp
            cupY (Cup): Cup tham gia kết hợp

        Returns:
           Cup: CupXY
        """
        combined_cup = cupX.combine_with(cupY)
        combined_cup.exp_sup = round(combined_cup.exp_sup, 2)  # Round after combining
        return combined_cup

    def get_first_UHUFP(self, min_util: int, k: int):
        """
        Lấy danh sách top k đầu tiên và danh sách ứng viên tham gia

        Args:
            min_util (int): ngưỡng tiện ích
            k (int): số lượng mẫu cần tìm

        Returns:
            List[Cup]: danh sách các ứng viên
        """
        candidate_list = []
        cups = sorted(self.single_cup.values(), key=lambda x: x.exp_sup, reverse=True)
        # chỉ lấy k Cup đầu tiên
        for cup in cups[:k]:
            if cup.trans_wei_util >= min_util:
                candidate_list.append(cup)
            if cup.utility >= min_util:
                self.top_UHUFP.append({'name': cup.name, 'exp_sup': cup.exp_sup, 'utility': cup.utility})
        return candidate_list


    def TUHUFPSearchHelper(self, combined, k: int):
        """
        Kiểm tra mẫu có phải là mẫu tiện ích cao hay không và thêm vào top k

        Args:
            combined (Cup): mẫu cần kiểm tra
            k (int): số lượng mẫu cần tìm
        """
        if combined.utility >= self.min_util:
            self.top_UHUFP.append({'name': combined.name, 'exp_sup': combined.exp_sup, 'utility': combined.utility})
            self.top_UHUFP.sort(key=lambda x: x['exp_sup'], reverse=True)
            if len(self.top_UHUFP) > k:
                self.top_UHUFP.pop()
                self.threshold = self.top_UHUFP[-1]['exp_sup']
            if len(self.top_UHUFP) == k:
                self.threshold = self.top_UHUFP[-1]['exp_sup']
        # print(f"Top {k} CUPs after add {combined}")
        # for cup in self.top_UHUFP[:]:
        #     print(cup)
        # print(self.threshold)


    def TUHUFPSearch(self, currentCup, k: int):
        """
        Phương thức chính của thuật toán, khai thác các mẫu UHUFP

        Args:
            currentCup (List[Cup]): danh sách ứng viên tham gia
            k (int): số lượng mẫu cần tìm
        """
        if len(currentCup) <= 1:
            # dừng thuật toán khi không còn ứng viên tham gia
            return
        # duyệt từng ứng viên
        for i in range(len(currentCup) - 1):
            newCupList = []
            for j in range(i + 1, len(currentCup)):
                overestimate = currentCup[i].exp_sup * currentCup[j].max_prob
                if overestimate < self.threshold:
                    break
                combined = self.combine_cup(currentCup[i], currentCup[j])
                # kiểm tra và thêm vào top k nếu thỏa điều kiện
                if combined.exp_sup > self.threshold:
                    self.TUHUFPSearchHelper(combined, k)
                    # thêm các ứng viên mới
                    if combined.trans_wei_util >= self.min_util:
                        newCupList.append(combined)
                        self.candidates += 1
            # tìm kiếm với danh sách ứng viên mới
            self.TUHUFPSearch(newCupList, k)


    def run_TUHUFP_algorithm(self, file_path: str, percentage, k: int):
        """
        Chạy thuật toán

        Args:
            file_path (str): đường dẫn tới file cơ sở dữ liệu
            percentage (_type_): ngưỡng tiện ích (%)
            k (int): số lượng mẫu tham gia
        """
        self.start_timestamp = time.time()
        self.candidates = 0
        self.database_size = 0

        self.read_data(file_path, percentage, k)
        print({'database util: ': self.database_util})

        # danh sách ứng viên
        candidate_list = self.get_first_UHUFP(self.min_util, k)
        if not self.single_cup:
            print("No CUP List was created. STOP ALGORITHM !!!")
            sys.exit(0)
        self.candidates = len(candidate_list)
        # print(self.candidates)

        # đặt threhold
        if len(self.top_UHUFP) == k:
            self.threshold = self.top_UHUFP[-1]['exp_sup']

        # print(f"Top {k} CUPs after sorting by exp_sup:")
        # i = 1
        # for cup in candidate_list[:]:
        #     print(f" {i}: {cup}")
        #     i +=1

        # Bắt đầu theo dõi bộ nhớ
        tracemalloc.start()
        # bắt đầu tìm kiếm
        self.TUHUFPSearch(candidate_list, k)
        print(f"Top {k} CUPs:")
        i = 1
        self.top_UHUFP.sort(key=lambda x: x['utility'], reverse=True)
        for cup in self.top_UHUFP[:]:
            print(f" {i}: {cup}")
            i +=1
        # Kết thúc theo dõi bộ nhớ và lấy thông tin
        _, self.peak_memory_usage = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        self.peak_memory_usage /= 10**6  # Chuyển đổi sang MB

        self.end_timestamp = time.time()

    def print_stats(self, path: str):
        """
        Thống kê thông tin kết quả chạy thuật toán

        Args:
            path (str): đường dẫn lưu thông tin
        """

        total_utility = 0
        directory = os.path.dirname(path)
        if directory and not os.path.exists(directory):
            os.makedirs(directory)

        with open(path, 'w') as output_file:
            output_file.write(f"minUtil: {self.min_util}\n")
            self.top_UHUFP.sort(key=lambda x: x['utility'], reverse=True)
            for t in self.top_UHUFP:
                total_utility += t['utility']
                output_file.write(f"{t['name']}: {t['exp_sup']}: {t['utility']}\n")
            output_file.write("=============  TOP-K UFPs v1.20 - STATS =============\n")
            output_file.write(f" Transactions count from database : {self.database_size}\n")
            output_file.write(f" Candidates count : {self.candidates}\n")
            output_file.write(f" Sum utility : {total_utility}\n")
            output_file.write(f" Algorithm run time : {self.end_timestamp - self.start_timestamp:.0f} seconds\n")
            output_file.write(f" Peak memory usage : {self.peak_memory_usage:.2f} MB\n")  # Ghi lại peak memory usage


## Dữ liệu thực nghiệm

In [ ]:
#chạy với Example
algorithm = AlgorithmTUHUFP()
algorithm.run_TUHUFP_algorithm("./example.txt, ./example_utility.txt", 0.3, 6)
algorithm.print_stats("../out/TUHUFP/output_example.txt")

Reading data . . .
Minimum utility set to: 30
{'database util: ': 102}
Top 6 CUPs:
 1: {'name': 'c, d, a', 'exp_sup': 0.98, 'utility': 42}
 2: {'name': 'c, d, b', 'exp_sup': 0.88, 'utility': 40}
 3: {'name': 'c, d', 'exp_sup': 1.95, 'utility': 38}
 4: {'name': 'd, a', 'exp_sup': 1.2, 'utility': 35}
 5: {'name': 'd, a, b', 'exp_sup': 0.55, 'utility': 33}
 6: {'name': 'd', 'exp_sup': 3.3, 'utility': 30}


## Hiện thực foodmart

In [ ]:
algorithm = AlgorithmTUHUFP()
algorithm.run_TUHUFP_algorithm("./input_foodmart.txt, ./foodmart_utility.txt", 0.0004, 100)
algorithm.print_stats("../out/TUHUFP/foodmart/output_foodmart_top_100.txt")

Reading data . . .
Minimum utility set to: 4804
{'database util: ': 12011023}
Top 100 CUPs:
 1: {'name': '1373', 'exp_sup': 13.55, 'utility': 25560}
 2: {'name': '1292', 'exp_sup': 12.6, 'utility': 24642}
 3: {'name': '988', 'exp_sup': 9.3, 'utility': 24187}
 4: {'name': '222', 'exp_sup': 9.33, 'utility': 22910}
 5: {'name': '272', 'exp_sup': 13.54, 'utility': 22678}
 6: {'name': '1426', 'exp_sup': 9.96, 'utility': 21692}
 7: {'name': '1161', 'exp_sup': 10.54, 'utility': 21540}
 8: {'name': '1045', 'exp_sup': 9.48, 'utility': 21090}
 9: {'name': '1012', 'exp_sup': 11.65, 'utility': 19861}
 10: {'name': '175', 'exp_sup': 9.14, 'utility': 19635}
 11: {'name': '1508', 'exp_sup': 10.16, 'utility': 19116}
 12: {'name': '276', 'exp_sup': 10.37, 'utility': 18528}
 13: {'name': '278', 'exp_sup': 9.33, 'utility': 18480}
 14: {'name': '918', 'exp_sup': 11.38, 'utility': 18414}
 15: {'name': '1282', 'exp_sup': 10.05, 'utility': 18232}
 16: {'name': '1110', 'exp_sup': 10.1, 'utility': 17920}
 17: 

In [7]:
algorithm = AlgorithmTUHUFP()
algorithm.run_TUHUFP_algorithm("./input_foodmart.txt, ./foodmart_utility.txt", 0.0004, 300)
algorithm.print_stats("..out/TUHUFP/foodmart/output_foodmart_top_300.txt")

Reading data . . .
Minimum utility set to: 4804
{'database util: ': 12011023}
Top 300 CUPs:
 1: {'name': '1373', 'exp_sup': 13.55, 'utility': 25560}
 2: {'name': '1292', 'exp_sup': 12.6, 'utility': 24642}
 3: {'name': '988', 'exp_sup': 9.3, 'utility': 24187}
 4: {'name': '225', 'exp_sup': 8.04, 'utility': 22951}
 5: {'name': '222', 'exp_sup': 9.33, 'utility': 22910}
 6: {'name': '272', 'exp_sup': 13.54, 'utility': 22678}
 7: {'name': '1426', 'exp_sup': 9.96, 'utility': 21692}
 8: {'name': '1161', 'exp_sup': 10.54, 'utility': 21540}
 9: {'name': '1045', 'exp_sup': 9.48, 'utility': 21090}
 10: {'name': '1518', 'exp_sup': 7.94, 'utility': 20416}
 11: {'name': '1180', 'exp_sup': 8.06, 'utility': 19890}
 12: {'name': '1012', 'exp_sup': 11.65, 'utility': 19861}
 13: {'name': '175', 'exp_sup': 9.14, 'utility': 19635}
 14: {'name': '1380', 'exp_sup': 7.99, 'utility': 19548}
 15: {'name': '1368', 'exp_sup': 7.57, 'utility': 19525}
 16: {'name': '1508', 'exp_sup': 10.16, 'utility': 19116}
 17: {

In [ ]:
algorithm = AlgorithmTUHUFP()
algorithm.run_TUHUFP_algorithm("./input_foodmart.txt, ./foodmart_utility.txt", 0.0004, 500)
algorithm.print_stats("../out/TUHUFP/foodmart/output_foodmart_top_500.txt")

Reading data . . .
Minimum utility set to: 4804
{'database util: ': 12011023}
Top 500 CUPs:
 1: {'name': '1373', 'exp_sup': 13.55, 'utility': 25560}
 2: {'name': '1292', 'exp_sup': 12.6, 'utility': 24642}
 3: {'name': '988', 'exp_sup': 9.3, 'utility': 24187}
 4: {'name': '225', 'exp_sup': 8.04, 'utility': 22951}
 5: {'name': '222', 'exp_sup': 9.33, 'utility': 22910}
 6: {'name': '272', 'exp_sup': 13.54, 'utility': 22678}
 7: {'name': '1426', 'exp_sup': 9.96, 'utility': 21692}
 8: {'name': '1161', 'exp_sup': 10.54, 'utility': 21540}
 9: {'name': '1045', 'exp_sup': 9.48, 'utility': 21090}
 10: {'name': '446', 'exp_sup': 6.93, 'utility': 20625}
 11: {'name': '83', 'exp_sup': 6.68, 'utility': 20563}
 12: {'name': '1518', 'exp_sup': 7.94, 'utility': 20416}
 13: {'name': '1180', 'exp_sup': 8.06, 'utility': 19890}
 14: {'name': '1012', 'exp_sup': 11.65, 'utility': 19861}
 15: {'name': '175', 'exp_sup': 9.14, 'utility': 19635}
 16: {'name': '1380', 'exp_sup': 7.99, 'utility': 19548}
 17: {'nam

In [ ]:
algorithm = AlgorithmTUHUFP()
algorithm.run_TUHUFP_algorithm("./input_foodmart.txt, ./foodmart_utility.txt", 0.0004, 700)
algorithm.print_stats("../out/TUHUFP/foodmart/output_foodmart_top_700.txt")

Reading data . . .
Minimum utility set to: 4804
{'database util: ': 12011023}
Top 700 CUPs:
 1: {'name': '1373', 'exp_sup': 13.55, 'utility': 25560}
 2: {'name': '1292', 'exp_sup': 12.6, 'utility': 24642}
 3: {'name': '988', 'exp_sup': 9.3, 'utility': 24187}
 4: {'name': '225', 'exp_sup': 8.04, 'utility': 22951}
 5: {'name': '222', 'exp_sup': 9.33, 'utility': 22910}
 6: {'name': '272', 'exp_sup': 13.54, 'utility': 22678}
 7: {'name': '1426', 'exp_sup': 9.96, 'utility': 21692}
 8: {'name': '1161', 'exp_sup': 10.54, 'utility': 21540}
 9: {'name': '1045', 'exp_sup': 9.48, 'utility': 21090}
 10: {'name': '446', 'exp_sup': 6.93, 'utility': 20625}
 11: {'name': '83', 'exp_sup': 6.68, 'utility': 20563}
 12: {'name': '1518', 'exp_sup': 7.94, 'utility': 20416}
 13: {'name': '1180', 'exp_sup': 8.06, 'utility': 19890}
 14: {'name': '1012', 'exp_sup': 11.65, 'utility': 19861}
 15: {'name': '175', 'exp_sup': 9.14, 'utility': 19635}
 16: {'name': '1380', 'exp_sup': 7.99, 'utility': 19548}
 17: {'nam

In [ ]:
algorithm = AlgorithmTUHUFP()
algorithm.run_TUHUFP_algorithm("./input_foodmart.txt, ./foodmart_utility.txt", 0.0004, 900)
algorithm.print_stats("../out/TUHUFP/foodmart/output_foodmart_top_900.txt")

Reading data . . .
Minimum utility set to: 4804
{'database util: ': 12011023}
Top 900 CUPs:
 1: {'name': '1373', 'exp_sup': 13.55, 'utility': 25560}
 2: {'name': '1292', 'exp_sup': 12.6, 'utility': 24642}
 3: {'name': '988', 'exp_sup': 9.3, 'utility': 24187}
 4: {'name': '225', 'exp_sup': 8.04, 'utility': 22951}
 5: {'name': '222', 'exp_sup': 9.33, 'utility': 22910}
 6: {'name': '272', 'exp_sup': 13.54, 'utility': 22678}
 7: {'name': '1426', 'exp_sup': 9.96, 'utility': 21692}
 8: {'name': '1161', 'exp_sup': 10.54, 'utility': 21540}
 9: {'name': '1045', 'exp_sup': 9.48, 'utility': 21090}
 10: {'name': '446', 'exp_sup': 6.93, 'utility': 20625}
 11: {'name': '83', 'exp_sup': 6.68, 'utility': 20563}
 12: {'name': '1518', 'exp_sup': 7.94, 'utility': 20416}
 13: {'name': '1180', 'exp_sup': 8.06, 'utility': 19890}
 14: {'name': '1012', 'exp_sup': 11.65, 'utility': 19861}
 15: {'name': '175', 'exp_sup': 9.14, 'utility': 19635}
 16: {'name': '1380', 'exp_sup': 7.99, 'utility': 19548}
 17: {'nam

## Hiện thực Chess

In [ ]:
# chạy với chess
algorithm = AlgorithmTUHUFP()
algorithm.run_TUHUFP_algorithm("./input_chess.txt, ./chess_utility.txt", 0.001, 100)
algorithm.print_stats("../out/TUHUFP/chess/output_chess_top_100.txt")

Reading data . . .
Minimum utility set to: 2156
{'database util: ': 2156659}
Top 100 CUPs:
 1: {'name': '52, 40', 'exp_sup': 800.33, 'utility': 290726}
 2: {'name': '40, 48', 'exp_sup': 741.87, 'utility': 290391}
 3: {'name': '40, 34', 'exp_sup': 737.74, 'utility': 277239}
 4: {'name': '29, 40', 'exp_sup': 792.51, 'utility': 255631}
 5: {'name': '40, 60', 'exp_sup': 773.52, 'utility': 253428}
 6: {'name': '40, 66', 'exp_sup': 734.99, 'utility': 243649}
 7: {'name': '40, 56', 'exp_sup': 746.88, 'utility': 243050}
 8: {'name': '40, 7', 'exp_sup': 758.2, 'utility': 231271}
 9: {'name': '52, 48', 'exp_sup': 761.78, 'utility': 212460}
 10: {'name': '58, 40', 'exp_sup': 790.04, 'utility': 205017}
 11: {'name': '40, 36', 'exp_sup': 761.32, 'utility': 198648}
 12: {'name': '52, 34', 'exp_sup': 750.57, 'utility': 197472}
 13: {'name': '40, 62', 'exp_sup': 764.28, 'utility': 197416}
 14: {'name': '40', 'exp_sup': 1579.27, 'utility': 188760}
 15: {'name': '29, 48', 'exp_sup': 744.94, 'utility': 1

In [ ]:
# chạy với chess
algorithm = AlgorithmTUHUFP()
algorithm.run_TUHUFP_algorithm("./input_chess.txt, ./chess_utility.txt", 0.001, 300)
algorithm.print_stats("../out/TUHUFP/chess/output_chess_top_300.txt")

Reading data . . .
Minimum utility set to: 2156
{'database util: ': 2156659}
Top 300 CUPs:
 1: {'name': '52, 40', 'exp_sup': 800.33, 'utility': 290726}
 2: {'name': '40, 48', 'exp_sup': 741.87, 'utility': 290391}
 3: {'name': '40, 34', 'exp_sup': 737.74, 'utility': 277239}
 4: {'name': '29, 40', 'exp_sup': 792.51, 'utility': 255631}
 5: {'name': '40, 60', 'exp_sup': 773.52, 'utility': 253428}
 6: {'name': '40, 66', 'exp_sup': 734.99, 'utility': 243649}
 7: {'name': '40, 56', 'exp_sup': 746.88, 'utility': 243050}
 8: {'name': '40, 7', 'exp_sup': 758.2, 'utility': 231271}
 9: {'name': '40, 3', 'exp_sup': 693.84, 'utility': 226522}
 10: {'name': '40, 5', 'exp_sup': 718.05, 'utility': 223282}
 11: {'name': '40, 72', 'exp_sup': 582.56, 'utility': 214715}
 12: {'name': '52, 48', 'exp_sup': 761.78, 'utility': 212460}
 13: {'name': '40, 64', 'exp_sup': 671.11, 'utility': 212460}
 14: {'name': '58, 40', 'exp_sup': 790.04, 'utility': 205017}
 15: {'name': '48, 34', 'exp_sup': 699.74, 'utility': 

In [ ]:
# chạy với chess
algorithm = AlgorithmTUHUFP()
algorithm.run_TUHUFP_algorithm("./input_chess.txt, ./chess_utility.txt", 0.001, 500)
algorithm.print_stats("../out/TUHUFP/chess/output_chess_top_500.txt")

Reading data . . .
Minimum utility set to: 2156
{'database util: ': 2156659}
Top 500 CUPs:
 1: {'name': '52, 40', 'exp_sup': 800.33, 'utility': 290726}
 2: {'name': '40, 48', 'exp_sup': 741.87, 'utility': 290391}
 3: {'name': '40, 34', 'exp_sup': 737.74, 'utility': 277239}
 4: {'name': '29, 40', 'exp_sup': 792.51, 'utility': 255631}
 5: {'name': '40, 60', 'exp_sup': 773.52, 'utility': 253428}
 6: {'name': '40, 66', 'exp_sup': 734.99, 'utility': 243649}
 7: {'name': '40, 56', 'exp_sup': 746.88, 'utility': 243050}
 8: {'name': '40, 7', 'exp_sup': 758.2, 'utility': 231271}
 9: {'name': '40, 3', 'exp_sup': 693.84, 'utility': 226522}
 10: {'name': '40, 5', 'exp_sup': 718.05, 'utility': 223282}
 11: {'name': '40, 72', 'exp_sup': 582.56, 'utility': 214715}
 12: {'name': '52, 48', 'exp_sup': 761.78, 'utility': 212460}
 13: {'name': '40, 64', 'exp_sup': 671.11, 'utility': 212460}
 14: {'name': '58, 40', 'exp_sup': 790.04, 'utility': 205017}
 15: {'name': '48, 34', 'exp_sup': 699.74, 'utility': 

In [ ]:
# chạy với chess
algorithm = AlgorithmTUHUFP()
algorithm.run_TUHUFP_algorithm("./input_chess.txt, ./chess_utility.txt", 0.001, 700)
algorithm.print_stats("../out/TUHUFP/chess/output_chess_top_700.txt")

Reading data . . .
Minimum utility set to: 2156
{'database util: ': 2156659}
Top 700 CUPs:
 1: {'name': '52, 40, 48', 'exp_sup': 378.53, 'utility': 385381}
 2: {'name': '52, 40, 34', 'exp_sup': 373.87, 'utility': 373506}
 3: {'name': '52, 29, 40', 'exp_sup': 398.51, 'utility': 356097}
 4: {'name': '52, 40, 60', 'exp_sup': 394.11, 'utility': 352926}
 5: {'name': '52, 40, 66', 'exp_sup': 373.61, 'utility': 339395}
 6: {'name': '52, 40, 56', 'exp_sup': 382.07, 'utility': 339206}
 7: {'name': '52, 40, 7', 'exp_sup': 383.08, 'utility': 328740}
 8: {'name': '29, 40, 60', 'exp_sup': 384.66, 'utility': 318464}
 9: {'name': '52, 58, 40', 'exp_sup': 401.39, 'utility': 306482}
 10: {'name': '29, 40, 56', 'exp_sup': 373.12, 'utility': 305344}
 11: {'name': '58, 40, 48', 'exp_sup': 372.95, 'utility': 305339}
 12: {'name': '52, 40, 36', 'exp_sup': 382.28, 'utility': 296932}
 13: {'name': '29, 40, 7', 'exp_sup': 381.05, 'utility': 296027}
 14: {'name': '52, 40, 62', 'exp_sup': 391.2, 'utility': 29454

In [ ]:
# chạy với chess
algorithm = AlgorithmTUHUFP()
algorithm.run_TUHUFP_algorithm("./input_chess.txt, ./chess_utility.txt", 0.001, 900)
algorithm.print_stats("../out/TUHUFP/chess/output_chess_top_900.txt")

Reading data . . .
Minimum utility set to: 2156
{'database util: ': 2156659}
Top 900 CUPs:
 1: {'name': '52, 40, 48', 'exp_sup': 378.53, 'utility': 385381}
 2: {'name': '52, 40, 34', 'exp_sup': 373.87, 'utility': 373506}
 3: {'name': '52, 29, 40', 'exp_sup': 398.51, 'utility': 356097}
 4: {'name': '52, 40, 60', 'exp_sup': 394.11, 'utility': 352926}
 5: {'name': '29, 40, 48', 'exp_sup': 366.24, 'utility': 351549}
 6: {'name': '40, 60, 48', 'exp_sup': 366.71, 'utility': 351142}
 7: {'name': '29, 40, 34', 'exp_sup': 368.8, 'utility': 340850}
 8: {'name': '52, 40, 66', 'exp_sup': 373.61, 'utility': 339395}
 9: {'name': '52, 40, 56', 'exp_sup': 382.07, 'utility': 339206}
 10: {'name': '40, 60, 34', 'exp_sup': 363.98, 'utility': 336770}
 11: {'name': '52, 40, 7', 'exp_sup': 383.08, 'utility': 328740}
 12: {'name': '40, 7, 48', 'exp_sup': 358.06, 'utility': 325169}
 13: {'name': '29, 40, 60', 'exp_sup': 384.66, 'utility': 318464}
 14: {'name': '52, 40, 5', 'exp_sup': 368.71, 'utility': 317307

## Hiện thực retail

In [ ]:
# chạy với retail
algorithm = AlgorithmTUHUFP()
algorithm.run_TUHUFP_algorithm("./input_retail.txt, ./retail_utility.txt", 0.0001, 100)
algorithm.print_stats("../out/TUHUFP/retail/output_retail_top_100.txt")

Reading data . . .
Minimum utility set to: 1491
{'database util: ': 14910915}
Top 100 CUPs:
 1: {'name': '40, 49', 'exp_sup': 7297.73, 'utility': 476144}
 2: {'name': '49', 'exp_sup': 21135.04, 'utility': 461182}
 3: {'name': '40', 'exp_sup': 25268.8, 'utility': 277044}
 4: {'name': '33', 'exp_sup': 7519.53, 'utility': 250500}
 5: {'name': '40, 42', 'exp_sup': 2824.5, 'utility': 247864}
 6: {'name': '49, 42', 'exp_sup': 2261.59, 'utility': 245579}
 7: {'name': '42', 'exp_sup': 7426.48, 'utility': 245544}
 8: {'name': '40, 49, 42', 'exp_sup': 910.15, 'utility': 238995}
 9: {'name': '49, 33', 'exp_sup': 1998.65, 'utility': 219766}
 10: {'name': '40, 33', 'exp_sup': 2084.63, 'utility': 184488}
 11: {'name': '40, 49, 33', 'exp_sup': 670.37, 'utility': 176116}
 12: {'name': '49, 39', 'exp_sup': 1978.64, 'utility': 173210}
 13: {'name': '39', 'exp_sup': 7775.43, 'utility': 169166}
 14: {'name': '40, 39', 'exp_sup': 2602.65, 'utility': 168244}
 15: {'name': '40, 49, 39', 'exp_sup': 763.78, 'u

In [ ]:
# chạy với retail
algorithm = AlgorithmTUHUFP()
algorithm.run_TUHUFP_algorithm("./input_retail.txt, ./retail_utility.txt", 0.0001, 300)
algorithm.print_stats("../out/TUHUFP/retail/output_retail_top_300.txt")

Reading data . . .
Minimum utility set to: 1491
{'database util: ': 14910915}
Top 300 CUPs:
 1: {'name': '40, 49', 'exp_sup': 7297.73, 'utility': 476144}
 2: {'name': '49', 'exp_sup': 21135.04, 'utility': 461182}
 3: {'name': '40', 'exp_sup': 25268.8, 'utility': 277044}
 4: {'name': '33', 'exp_sup': 7519.53, 'utility': 250500}
 5: {'name': '40, 42', 'exp_sup': 2824.5, 'utility': 247864}
 6: {'name': '49, 42', 'exp_sup': 2261.59, 'utility': 245579}
 7: {'name': '42', 'exp_sup': 7426.48, 'utility': 245544}
 8: {'name': '40, 49, 42', 'exp_sup': 910.15, 'utility': 238995}
 9: {'name': '49, 33', 'exp_sup': 1998.65, 'utility': 219766}
 10: {'name': '40, 33', 'exp_sup': 2084.63, 'utility': 184488}
 11: {'name': '40, 49, 33', 'exp_sup': 670.37, 'utility': 176116}
 12: {'name': '49, 39', 'exp_sup': 1978.64, 'utility': 173210}
 13: {'name': '39', 'exp_sup': 7775.43, 'utility': 169166}
 14: {'name': '40, 39', 'exp_sup': 2602.65, 'utility': 168244}
 15: {'name': '40, 49, 39', 'exp_sup': 763.78, 'u

In [ ]:
# chạy với retail
algorithm = AlgorithmTUHUFP()
algorithm.run_TUHUFP_algorithm("./input_retail.txt, ./retail_utility.txt", 0.0001, 500)
algorithm.print_stats("../out/TUHUFP/retail/output_retail_top_500.txt")

Reading data . . .
Minimum utility set to: 1491
{'database util: ': 14910915}
Top 500 CUPs:
 1: {'name': '40, 49', 'exp_sup': 7297.73, 'utility': 476144}
 2: {'name': '49', 'exp_sup': 21135.04, 'utility': 461182}
 3: {'name': '40', 'exp_sup': 25268.8, 'utility': 277044}
 4: {'name': '33', 'exp_sup': 7519.53, 'utility': 250500}
 5: {'name': '40, 42', 'exp_sup': 2824.5, 'utility': 247864}
 6: {'name': '49, 42', 'exp_sup': 2261.59, 'utility': 245579}
 7: {'name': '42', 'exp_sup': 7426.48, 'utility': 245544}
 8: {'name': '40, 49, 42', 'exp_sup': 910.15, 'utility': 238995}
 9: {'name': '49, 33', 'exp_sup': 1998.65, 'utility': 219766}
 10: {'name': '40, 33', 'exp_sup': 2084.63, 'utility': 184488}
 11: {'name': '40, 49, 33', 'exp_sup': 670.37, 'utility': 176116}
 12: {'name': '49, 39', 'exp_sup': 1978.64, 'utility': 173210}
 13: {'name': '39', 'exp_sup': 7775.43, 'utility': 169166}
 14: {'name': '40, 39', 'exp_sup': 2602.65, 'utility': 168244}
 15: {'name': '40, 49, 39', 'exp_sup': 763.78, 'u

In [ ]:
# chạy với retail
algorithm = AlgorithmTUHUFP()
algorithm.run_TUHUFP_algorithm("./input_retail.txt, ./retail_utility.txt", 0.0001, 700)
algorithm.print_stats("../out/TUHUFP/retail/output_retail_top_700.txt")

Reading data . . .
Minimum utility set to: 1491
{'database util: ': 14910915}
Top 700 CUPs:
 1: {'name': '40, 49', 'exp_sup': 7297.73, 'utility': 476144}
 2: {'name': '49', 'exp_sup': 21135.04, 'utility': 461182}
 3: {'name': '40', 'exp_sup': 25268.8, 'utility': 277044}
 4: {'name': '33', 'exp_sup': 7519.53, 'utility': 250500}
 5: {'name': '40, 42', 'exp_sup': 2824.5, 'utility': 247864}
 6: {'name': '49, 42', 'exp_sup': 2261.59, 'utility': 245579}
 7: {'name': '42', 'exp_sup': 7426.48, 'utility': 245544}
 8: {'name': '40, 49, 42', 'exp_sup': 910.15, 'utility': 238995}
 9: {'name': '49, 33', 'exp_sup': 1998.65, 'utility': 219766}
 10: {'name': '40, 33', 'exp_sup': 2084.63, 'utility': 184488}
 11: {'name': '40, 49, 33', 'exp_sup': 670.37, 'utility': 176116}
 12: {'name': '49, 39', 'exp_sup': 1978.64, 'utility': 173210}
 13: {'name': '39', 'exp_sup': 7775.43, 'utility': 169166}
 14: {'name': '40, 39', 'exp_sup': 2602.65, 'utility': 168244}
 15: {'name': '40, 49, 39', 'exp_sup': 763.78, 'u

In [ ]:
# chạy với retail
algorithm = AlgorithmTUHUFP()
algorithm.run_TUHUFP_algorithm("./input_retail.txt, ./retail_utility.txt", 0.0001, 900)
algorithm.print_stats("../out/TUHUFP/retail/output_retail_top_900.txt")

Reading data . . .
Minimum utility set to: 1491
{'database util: ': 14910915}
Top 900 CUPs:
 1: {'name': '40, 49', 'exp_sup': 7297.73, 'utility': 476144}
 2: {'name': '49', 'exp_sup': 21135.04, 'utility': 461182}
 3: {'name': '40', 'exp_sup': 25268.8, 'utility': 277044}
 4: {'name': '33', 'exp_sup': 7519.53, 'utility': 250500}
 5: {'name': '40, 42', 'exp_sup': 2824.5, 'utility': 247864}
 6: {'name': '49, 42', 'exp_sup': 2261.59, 'utility': 245579}
 7: {'name': '42', 'exp_sup': 7426.48, 'utility': 245544}
 8: {'name': '40, 49, 42', 'exp_sup': 910.15, 'utility': 238995}
 9: {'name': '49, 33', 'exp_sup': 1998.65, 'utility': 219766}
 10: {'name': '40, 33', 'exp_sup': 2084.63, 'utility': 184488}
 11: {'name': '40, 49, 33', 'exp_sup': 670.37, 'utility': 176116}
 12: {'name': '49, 39', 'exp_sup': 1978.64, 'utility': 173210}
 13: {'name': '39', 'exp_sup': 7775.43, 'utility': 169166}
 14: {'name': '40, 39', 'exp_sup': 2602.65, 'utility': 168244}
 15: {'name': '40, 49, 39', 'exp_sup': 763.78, 'u

# Thuật toán 1: SKYFUP

In [ ]:
import sys, os
import time
import tracemalloc

## Cài đặt cấu trúc dữ liệu

### Cài đặt UP-Lists

In [ ]:
class Up:
    """
    Đối tượng lưu trữ các thông tin của i ứng với giao dịch (TID, xác xuất, tiện ích, tiện ích còn lại, tiện ích giao dịch)
    """
    def __init__(self, TID: int, prob: float, util: int, r_utility: int, trans_util: int):
        """
        Khởi tạo UP

        Args:
            TID (int): mã giao dịch
            prob (float): xác xuất
            util (int): giá trị tiện ích
            r_utility (int): tiện ích còn lại
            trans_util (int): tiện ích giao dịch
        """
        self.TID = TID
        self.prob = round(prob, 2)
        self.util = util
        self.r_utility = r_utility
        self.trans_util = trans_util

    def combine_with(self, other_up):
        """
        Kết hợp 2 Up với nhau

        Args:
            other_up (Up): Up cần kết hợp

        Returns:
            Up: một Up mới
        """
        combined_prob = round(self.prob * other_up.prob, 2)
        combined_util = self.util + other_up.util
        return Up(self.TID, combined_prob, combined_util, other_up.r_utility, other_up.trans_util)

    def __repr__(self):
        return f"Up(TID={self.TID}, prob={self.prob}, util={self.util}, r_utility={self.r_utility}, trans_util={self.trans_util})\n"

### Cài đặt IMCUP-Lists

In [ ]:
class IMCUP:
    """
    Đối tượng lưu trữ các thông tin của một item bao gồm: tên, độ hỗ trợ mong đợi, danh sách Up,
        giá trị xác xuất lớn nhất trong Up, TWU, giá trị tiện ích, phần tử cuối của item
    """
    def __init__(self, name: str, up_list=None):
        """
        Khởi tạo IMCUP

        Args:
            name (str): tên của IMCUP hay tên của item
            up_list (List[Up], None): danh sách Up của item
        """
        self.name = name
        self.exp_sup = round(sum(up.prob for up in up_list), 2) if up_list else 0
        self.up_list = up_list if up_list else []
        self.max_prob = round(max(up.prob for up in up_list), 2) if up_list else 0
        self.trans_wei_util = sum(up.trans_util for up in up_list) if up_list else 0
        self.utility = sum(up.util for up in up_list) if up_list else 0
        self.r_utility = sum(up.r_utility for up in up_list) if up_list else 0
        self.last = []

    def update(self, probability: float, TID, trans_util, r_utility, util_value) -> None:
        """
        Cập nhật lại các giá trị trong IMCUP

        Args:
            probability (float): xác xuất tồn tại
            TID (_type_): mã giao dịch
            trans_util (_type_): tiện ích giao dịch
            r_utility (_type_): tiện ích còn lại
            util_value (_type_): giá trị tiện ích
        """
        probability = round(probability, 2)
        up = Up(TID, probability, util_value, r_utility, trans_util)
        self.exp_sup = round(self.exp_sup + probability, 2)
        self.utility += util_value
        self.r_utility += r_utility
        self.up_list.append(up)
        self.max_prob = max(self.max_prob, probability)
        self.trans_wei_util += trans_util

    def combine_up(self, up_list_x, tep_list_y):
        """
        Kết hợp hai danh sách Up

        Args:
            up_list_x (List[Up]): danh sách Up của item X
            tep_list_y (List[Up]): danh sách Up của item Y

        Returns:
            List[Up]: danh sách Up của item XY
        """
        up_list_xy = []
        i, j = 0, 0
        while i < len(up_list_x) and j < len(tep_list_y):
            tX = up_list_x[i]
            tY = tep_list_y[j]
            if tX.TID < tY.TID:
                i += 1
            elif tX.TID > tY.TID:
                j += 1
            else:
                combined_prob = round(tX.prob * tY.prob, 2)
                combined_util = tX.util + tY.util
                up_list_xy.append(Up(tX.TID, combined_prob, combined_util, tY.r_utility, tX.trans_util))
                i += 1
                j += 1
        return up_list_xy

    def combine_with(self, other_up):
        """
        Kết hợp hai IMCUP lại với nhau

        Args:
            other_up (IMCUP): IMCUP tham gia kết hợp

        Returns:
            IMCUP: một IMCUP mới
        """
        if len(other_up.last) == 0:
            combined_name = self.name + ', ' + other_up.name
            combined_tep_list = self.combine_up(self.up_list, other_up.up_list)
            last = [other_up]
        else:
            combined_name = self.name + ', ' + other_up.last[0].name
            combined_tep_list = self.combine_up(self.up_list, other_up.last[0].up_list)
            last = other_up.last

        combined_cup = IMCUP(combined_name, combined_tep_list)
        combined_cup.last = last
        return combined_cup

    def __repr__(self) -> str:
        return f"IMCUP(name={self.name}, exp_sup={self.exp_sup}, r_utility={self.r_utility}, utility={self.utility})"

## Cài đặt thuật toán

In [ ]:
class AlgorithmSKYFUP:
    """
    Triển khai thuật toán khai thác top-k mẫu phổ biến tiện ích cao không chắc chắn
    """
    def __init__(self):
        self.start_timestamp = 0
        self.end_timestamp = 0
        self.database_size = 0
        self.database_util = 0
        self.candidates = 0
        self.top_UHUFP = []
        self.single_imcup = {}
        self.threshold = float('-inf')
        self.min_util = float('-inf')
        self.peak_memory_usage = 0

    def read_data(self, file_path: str, percentage: float, k: int) -> None:
        """
        Đọc cơ sỡ dữ liệu từ file và xử lý dữ liệu

        Args:
            file_path (str): đường dẫn chứa file cơ sở dữ liệu
            percentage (float): ngưỡng tiện ích (%)
            k (int): số lượng mẫu cần tìm
        """

        file_paths = file_path.split(", ")

        try:
            with open(file_paths[0], 'r') as file1, open(file_paths[1], 'r') as file2:
                print("Reading data . . .")
                tlines = file1.readlines()
                ulines = file2.readlines()
                i_name = tlines[0].strip().split(" ")

                TID = 1
                # xử lý từng giao dịch
                for prob_line, util_line in zip(tlines[1:], ulines):
                    self.process_data(i_name, prob_line.strip(), util_line.strip(), TID)
                    TID += 1
                    self.database_size += 1

                self.min_util = int(self.database_util * percentage)
                print(f"Minimum utility set to: {self.min_util}")
        except FileNotFoundError as e:
            print(f"File not found: {e}")
            print("STOP ALGORITHM !!!")
            sys.exit(0)
        except Exception as e:
            print(f"An error occurred: {e}")
            sys.exit(0)


    def process_data(self, i_name, prob_line: float, util_line: int, TID: int) -> None:
        """
        Xử lý từng giao dịch và tạo IMCUP

        Args:
            i_name (List[str]): danh sách các item
            prob_line (float): danh sách các xác xuất của item
            util_line (int): danh sách tiện ích của item
            TID (int): mã giao dịch
        """

        prob_list = prob_line.split(" ")
        trans = util_line.split(":")
        item_util = trans[0].split(" ") # Danh sách các mục (items)
        trans_util = int(trans[1]) # Tổng lợi ích của giao dịch (transaction utility)
        util_list = trans[2].split(" ") # Danh sách lợi ích từng mục trong giao dịch

        i_util_list = {i: int(util) for i, util in zip(item_util, util_list)}

        self.database_util += trans_util

        for i, prob in enumerate(prob_list):
            prob = prob.strip()

            # Chỉ xử lý nếu giá trị xác suất tồn tại và lớn hơn 0
            if prob and float(prob) > 0:
                try:
                    # Lấy tên mục (item name) từ chỉ mục (index)
                    i = i_name[i]
                    probability = round(float(prob), 2)  # Round to 2 decimal places

                   # Lấy giá trị lợi ích của mục hiện tại
                    util_value = i_util_list[i]
                    imcup_name = i

                    # Xác định vị trí của mục trong danh sách các keys
                    keys = list(i_util_list.keys())
                    index = keys.index(imcup_name)

                    # Tính tổng lợi ích của các mục sau mục hiện tại
                    r = sum(i_util_list[k] for k in keys[index + 1:])

                    #kiểm tra imcup đã được tạo hay chưa
                    if imcup_name in self.single_imcup:
                        self.single_imcup[imcup_name].update(probability, TID, trans_util, r, util_value)
                    else:
                        new_up = Up(TID, probability, util_value, r, trans_util)
                        self.single_imcup[imcup_name] = IMCUP(imcup_name, [new_up])
                except ValueError as ve:
                    print(f"Error processing probability '{prob}': {ve}")
                    continue


    def combine_cup(self, upX, upY):
        """
        Kết hơp hai Up X và Y

        Args:
            upX (Up): Up tham gia kết hợp
            upY (Up): Up tham gia kết hợp

        Returns:
           Up: UpXY
        """
        combined_cup = upX.combine_with(upY)
        combined_cup.exp_sup = round(combined_cup.exp_sup, 2)
        return combined_cup



    def check(self, combined, k: int) -> None:
        """
        Kiểm tra mẫu có phải là mẫu tiện ích cao

        Args:
            combined (Up): mẫu cần kiểm tra
           TUHUFP k (int): số lượng mẫu cần tìm
        """

        # kiểm tra và thêm vào top k nếu thỏa điều kiện utility > min_util
        if combined.utility >= self.min_util:
            self.top_UHUFP.append({'name': combined.name, 'exp_sup': combined.exp_sup, 'utility': combined.utility})
            self.top_UHUFP.sort(key=lambda x: x['exp_sup'], reverse=True)
            if len(self.top_UHUFP) > k:
                self.top_UHUFP.pop()
                self.threshold = self.top_UHUFP[-1]['exp_sup']
            # cập nhật lại threshold khi top_UHUFP đã đủ k mẫu
            if len(self.top_UHUFP) == k:
                self.threshold = self.top_UHUFP[-1]['exp_sup']


    def Search(self, currentUp, k: int) -> None:
        """
        Khai thác mẫu từ danh sách ứng viên

        Args:
            currentUp (List[Up]): danh sách ứng viên tham gia
            k (int): số lượng mẫu cần tìm
        """
        if len(currentUp) <= 1:
            # dừng thuật toán khi không còn ứng viên tham gia
            return
        # duyệt từng ứng viên
        for i in range(len(currentUp) - 1):
            # kiểm tra điều kiện tiện ích
            if currentUp[i].utility + currentUp[i].r_utility > self.min_util:
                newCupList = []
                for j in range(i + 1, len(currentUp)):
                    overestimate = currentUp[i].exp_sup * currentUp[j].max_prob
                    if currentUp[j].exp_sup < self.threshold or overestimate < self.threshold:
                        break
                    combined = self.combine_cup(currentUp[i], currentUp[j])
                    # kiểm tra và thêm vào top k nếu thỏa điều kiện
                    if combined.exp_sup > self.threshold:
                        self.check(combined, k)
                        # thêm các ứng viên mới
                        if combined.trans_wei_util >= self.min_util:
                            newCupList.append(combined)
                            self.candidates += 1
                # tìm kiếm với danh sách ứng viên mới
                self.Search(newCupList, k)

    def create_candidate(self, min_util: int, k: int):
        """
        Tìm top k đầu tiên và các ứng viên

        Args:
            min_util (int): ngưỡng tiện ích
            k (int): số lượng mẫu cần tìm

        Returns:
            List[Up]: danh sách các ứng viên
        """
        candidate_list = []
        imcup = sorted(self.single_imcup.values(), key=lambda x: x.exp_sup, reverse=True)
        # lấy k Up đầu tiên có giá trị expSup lớn nhất
        for up in imcup[:k]:
            if up.trans_wei_util >= min_util:
                candidate_list.append(up)
            if up.utility >= min_util:
                self.top_UHUFP.append({'name': up.name, 'exp_sup': up.exp_sup, 'utility': up.utility})
        return candidate_list


    def run(self, file_path: str, percentage, k: int) -> None:
        """
        Khởi chạy thuật toán

        Args:
            file_path (str): đường dẫn tới file cơ sở dữ liệu
            percentage (_type_): ngưỡng tiện ích (%)
            k (int): số lượng mẫu cần tìm
        """

        self.start_timestamp = time.time()
        self.candidates = 0
        self.database_size = 0

        # đọc dữ liệu từ file
        self.read_data(file_path, percentage, k)
        print({'database util: ': self.database_util})

        # danh sách ứng viên
        candidate_list = self.create_candidate(self.min_util, k)
        if not self.single_imcup:
            print("No Up List was created. STOP ALGORITHM !!!")
            sys.exit(0)
        self.candidates = len(candidate_list)

        # đặt threhold
        if len(self.top_UHUFP) == k:
            self.threshold = self.top_UHUFP[-1]['exp_sup']

        # Bắt đầu theo dõi bộ nhớ
        tracemalloc.start()
        # bắt đầu tìm kiếm
        self.Search(candidate_list, k)
        print(f"Top {k} CUPs:")
        i = 1
        for Up in self.top_UHUFP[:]:
            print(f" {i}: {Up}")
            i +=1
        # Kết thúc theo dõi bộ nhớ và lấy thông tin
        _, self.peak_memory_usage = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        self.peak_memory_usage /= 10**6  # Chuyển đổi sang MB

        self.end_timestamp = time.time()

    def print_stats(self, path: str) -> None:
        """
        Thống kê thông tin kết quả chạy thuật toán

        Args:
            path (str): đường dẫn lưu thông tin
        """

        total_utility = 0
        directory = os.path.dirname(path)
        if directory and not os.path.exists(directory):
            os.makedirs(directory)

        with open(path, 'w') as output_file:
            output_file.write(f"minUtil: {self.min_util}\n")
            self.top_UHUFP.sort(key=lambda x: x['utility'], reverse=True)
            for t in self.top_UHUFP:
                total_utility += t['utility']
                output_file.write(f"{t['name']}: {t['exp_sup']}: {t['utility']}\n")
            output_file.write("=============  TOP-K UFPs v1.20 - STATS =============\n")
            output_file.write(f" Transactions count from database : {self.database_size}\n")
            output_file.write(f" Candidates count : {self.candidates}\n")
            output_file.write(f" Sum utility : {total_utility}\n")
            output_file.write(f" Algorithm run time : {self.end_timestamp - self.start_timestamp:.0f} seconds\n")
            output_file.write(f" Peak memory usage : {self.peak_memory_usage:.2f} MB\n")  # Ghi lại peak memory usage


In [ ]:
# a b c d e
# 0.6 0 0.8 0.5 0
# 0 0.7 0.4 0 0
# 0.6 0.6 0.9 0.8 0
# 0 0.5 0 0.8 0
# 0.9 0.7 0.9 0.6 0.8
# 0 0 0.9 0 0.8
# 0 0 0.4 0.9 0
# 0.6 0.8 0 0.8 0.5
# 0.6 0 0.5 0.3 0
# 0 0 0.6 0 0.7


# a d c:65:14 7 44
# b c:37:4 33
# a b d c:38:21 4 2 11
# b d:11:8 3
# e a b d c:49:9 7 6 5 22
# e c:58:36 22
# d c:23:1 22
# e a b d:61:36 21 2 2
# a d c:59:14 1 44
# e c:42:9 33


## Dữ liệu thực nghiệm

In [ ]:
# chạy với Example
algorithm = AlgorithmSKYFUP()
algorithm.run("./example.txt, ./example_utility.txt", 0.3, 6)
algorithm.print_stats("../out/SKYFUP/output_example.txt")

Reading data . . .
Minimum utility set to: 30
{'database util: ': 102}
Top 6 CUPs:
 1: {'name': 'd', 'exp_sup': 3.3, 'utility': 30}
 2: {'name': 'c, d', 'exp_sup': 1.95, 'utility': 38}
 3: {'name': 'd, a', 'exp_sup': 1.2, 'utility': 35}
 4: {'name': 'c, d, a', 'exp_sup': 0.98, 'utility': 42}
 5: {'name': 'c, d, b', 'exp_sup': 0.88, 'utility': 40}
 6: {'name': 'd, a, b', 'exp_sup': 0.55, 'utility': 33}


## Hiện thực foodmart

In [ ]:
algorithm = AlgorithmSKYFUP()
algorithm.run("./input_foodmart.txt, ./foodmart_utility.txt", 0.0004, 100)
algorithm.print_stats("../out/SKYFUP/foodmart/output_foodmart_top_100.txt")

Reading data . . .
Minimum utility set to: 4804
{'database util: ': 12011023}
Top 100 CUPs:
 1: {'name': '304', 'exp_sup': 14.23, 'utility': 13607}
 2: {'name': '1373', 'exp_sup': 13.55, 'utility': 25560}
 3: {'name': '272', 'exp_sup': 13.54, 'utility': 22678}
 4: {'name': '1293', 'exp_sup': 12.66, 'utility': 8352}
 5: {'name': '1292', 'exp_sup': 12.6, 'utility': 24642}
 6: {'name': '545', 'exp_sup': 12.1, 'utility': 9419}
 7: {'name': '357', 'exp_sup': 11.97, 'utility': 11520}
 8: {'name': '634', 'exp_sup': 11.88, 'utility': 11881}
 9: {'name': '617', 'exp_sup': 11.75, 'utility': 7527}
 10: {'name': '1423', 'exp_sup': 11.75, 'utility': 16116}
 11: {'name': '916', 'exp_sup': 11.68, 'utility': 15163}
 12: {'name': '1012', 'exp_sup': 11.65, 'utility': 19861}
 13: {'name': '903', 'exp_sup': 11.62, 'utility': 14616}
 14: {'name': '831', 'exp_sup': 11.39, 'utility': 8848}
 15: {'name': '888', 'exp_sup': 11.38, 'utility': 5280}
 16: {'name': '918', 'exp_sup': 11.38, 'utility': 18414}
 17: {'

In [ ]:
algorithm = AlgorithmSKYFUP()
algorithm.run("./input_foodmart.txt, ./foodmart_utility.txt", 0.0004, 300)
algorithm.print_stats("../out/SKYFUP/foodmart/output_foodmart_top_300.txt")

Reading data . . .
Minimum utility set to: 4804
{'database util: ': 12011023}
Top 300 CUPs:
 1: {'name': '304', 'exp_sup': 14.23, 'utility': 13607}
 2: {'name': '1373', 'exp_sup': 13.55, 'utility': 25560}
 3: {'name': '272', 'exp_sup': 13.54, 'utility': 22678}
 4: {'name': '1293', 'exp_sup': 12.66, 'utility': 8352}
 5: {'name': '1292', 'exp_sup': 12.6, 'utility': 24642}
 6: {'name': '545', 'exp_sup': 12.1, 'utility': 9419}
 7: {'name': '357', 'exp_sup': 11.97, 'utility': 11520}
 8: {'name': '634', 'exp_sup': 11.88, 'utility': 11881}
 9: {'name': '617', 'exp_sup': 11.75, 'utility': 7527}
 10: {'name': '1423', 'exp_sup': 11.75, 'utility': 16116}
 11: {'name': '916', 'exp_sup': 11.68, 'utility': 15163}
 12: {'name': '1012', 'exp_sup': 11.65, 'utility': 19861}
 13: {'name': '903', 'exp_sup': 11.62, 'utility': 14616}
 14: {'name': '831', 'exp_sup': 11.39, 'utility': 8848}
 15: {'name': '888', 'exp_sup': 11.38, 'utility': 5280}
 16: {'name': '918', 'exp_sup': 11.38, 'utility': 18414}
 17: {'

In [ ]:
algorithm = AlgorithmSKYFUP()
algorithm.run("./input_foodmart.txt, ./foodmart_utility.txt", 0.0004, 500)
algorithm.print_stats("../out/SKYFUP/foodmart/output_foodmart_top_500.txt")

Reading data . . .
Minimum utility set to: 4804
{'database util: ': 12011023}
Top 500 CUPs:
 1: {'name': '304', 'exp_sup': 14.23, 'utility': 13607}
 2: {'name': '1373', 'exp_sup': 13.55, 'utility': 25560}
 3: {'name': '272', 'exp_sup': 13.54, 'utility': 22678}
 4: {'name': '1293', 'exp_sup': 12.66, 'utility': 8352}
 5: {'name': '1292', 'exp_sup': 12.6, 'utility': 24642}
 6: {'name': '545', 'exp_sup': 12.1, 'utility': 9419}
 7: {'name': '357', 'exp_sup': 11.97, 'utility': 11520}
 8: {'name': '634', 'exp_sup': 11.88, 'utility': 11881}
 9: {'name': '617', 'exp_sup': 11.75, 'utility': 7527}
 10: {'name': '1423', 'exp_sup': 11.75, 'utility': 16116}
 11: {'name': '916', 'exp_sup': 11.68, 'utility': 15163}
 12: {'name': '1012', 'exp_sup': 11.65, 'utility': 19861}
 13: {'name': '903', 'exp_sup': 11.62, 'utility': 14616}
 14: {'name': '831', 'exp_sup': 11.39, 'utility': 8848}
 15: {'name': '888', 'exp_sup': 11.38, 'utility': 5280}
 16: {'name': '918', 'exp_sup': 11.38, 'utility': 18414}
 17: {'

In [ ]:
algorithm = AlgorithmSKYFUP()
algorithm.run("./input_foodmart.txt, ./foodmart_utility.txt", 0.0004, 700)
algorithm.print_stats("../out/SKYFUP/foodmart/output_foodmart_top_700.txt")

Reading data . . .
Minimum utility set to: 4804
{'database util: ': 12011023}
Top 700 CUPs:
 1: {'name': '304', 'exp_sup': 14.23, 'utility': 13607}
 2: {'name': '1373', 'exp_sup': 13.55, 'utility': 25560}
 3: {'name': '272', 'exp_sup': 13.54, 'utility': 22678}
 4: {'name': '1293', 'exp_sup': 12.66, 'utility': 8352}
 5: {'name': '1292', 'exp_sup': 12.6, 'utility': 24642}
 6: {'name': '545', 'exp_sup': 12.1, 'utility': 9419}
 7: {'name': '357', 'exp_sup': 11.97, 'utility': 11520}
 8: {'name': '634', 'exp_sup': 11.88, 'utility': 11881}
 9: {'name': '617', 'exp_sup': 11.75, 'utility': 7527}
 10: {'name': '1423', 'exp_sup': 11.75, 'utility': 16116}
 11: {'name': '916', 'exp_sup': 11.68, 'utility': 15163}
 12: {'name': '1012', 'exp_sup': 11.65, 'utility': 19861}
 13: {'name': '903', 'exp_sup': 11.62, 'utility': 14616}
 14: {'name': '831', 'exp_sup': 11.39, 'utility': 8848}
 15: {'name': '888', 'exp_sup': 11.38, 'utility': 5280}
 16: {'name': '918', 'exp_sup': 11.38, 'utility': 18414}
 17: {'

In [ ]:
algorithm = AlgorithmSKYFUP()
algorithm.run("./input_foodmart.txt, ./foodmart_utility.txt", 0.0004, 900)
algorithm.print_stats("../out/SKYFUP/foodmart/output_foodmart_top_900.txt")

Reading data . . .
Minimum utility set to: 4804
{'database util: ': 12011023}
Top 900 CUPs:
 1: {'name': '304', 'exp_sup': 14.23, 'utility': 13607}
 2: {'name': '1373', 'exp_sup': 13.55, 'utility': 25560}
 3: {'name': '272', 'exp_sup': 13.54, 'utility': 22678}
 4: {'name': '1293', 'exp_sup': 12.66, 'utility': 8352}
 5: {'name': '1292', 'exp_sup': 12.6, 'utility': 24642}
 6: {'name': '545', 'exp_sup': 12.1, 'utility': 9419}
 7: {'name': '357', 'exp_sup': 11.97, 'utility': 11520}
 8: {'name': '634', 'exp_sup': 11.88, 'utility': 11881}
 9: {'name': '617', 'exp_sup': 11.75, 'utility': 7527}
 10: {'name': '1423', 'exp_sup': 11.75, 'utility': 16116}
 11: {'name': '916', 'exp_sup': 11.68, 'utility': 15163}
 12: {'name': '1012', 'exp_sup': 11.65, 'utility': 19861}
 13: {'name': '903', 'exp_sup': 11.62, 'utility': 14616}
 14: {'name': '831', 'exp_sup': 11.39, 'utility': 8848}
 15: {'name': '888', 'exp_sup': 11.38, 'utility': 5280}
 16: {'name': '918', 'exp_sup': 11.38, 'utility': 18414}
 17: {'

## Hiện thực Chess

In [ ]:
# chạy với chess
algorithm = AlgorithmSKYFUP()
algorithm.run("./input_chess.txt, ./chess_utility.txt", 0.001, 100)
algorithm.print_stats("../out/SKYFUP/chess/output_chess_top_100.txt")

Reading data . . .
Minimum utility set to: 2156
{'database util: ': 2156659}
Top 100 CUPs:
 1: {'name': '52', 'exp_sup': 1616.54, 'utility': 104178}
 2: {'name': '29', 'exp_sup': 1598.92, 'utility': 70088}
 3: {'name': '58', 'exp_sup': 1595.58, 'utility': 17462}
 4: {'name': '40', 'exp_sup': 1579.27, 'utility': 188760}
 5: {'name': '60', 'exp_sup': 1565.64, 'utility': 68908}
 6: {'name': '62', 'exp_sup': 1537.33, 'utility': 16784}
 7: {'name': '7', 'exp_sup': 1535.24, 'utility': 50790}
 8: {'name': '36', 'exp_sup': 1530.49, 'utility': 16934}
 9: {'name': '56', 'exp_sup': 1513.93, 'utility': 66780}
 10: {'name': '48', 'exp_sup': 1500.86, 'utility': 115367}
 11: {'name': '66', 'exp_sup': 1498.92, 'utility': 66644}
 12: {'name': '34', 'exp_sup': 1493.64, 'utility': 99330}
 13: {'name': '5', 'exp_sup': 1466.73, 'utility': 48369}
 14: {'name': '9', 'exp_sup': 1446.46, 'utility': 31818}
 15: {'name': '25', 'exp_sup': 1445.46, 'utility': 15465}
 16: {'name': '3', 'exp_sup': 1407.44, 'utility'

In [ ]:
# chạy với chess
algorithm = AlgorithmSKYFUP()
algorithm.run("./input_chess.txt, ./chess_utility.txt", 0.001, 300)
algorithm.print_stats("../out/SKYFUP/chess/output_chess_top_300.txt")

Reading data . . .
Minimum utility set to: 2156
{'database util: ': 2156659}
Top 300 CUPs:
 1: {'name': '52', 'exp_sup': 1616.54, 'utility': 104178}
 2: {'name': '29', 'exp_sup': 1598.92, 'utility': 70088}
 3: {'name': '58', 'exp_sup': 1595.58, 'utility': 17462}
 4: {'name': '40', 'exp_sup': 1579.27, 'utility': 188760}
 5: {'name': '60', 'exp_sup': 1565.64, 'utility': 68908}
 6: {'name': '62', 'exp_sup': 1537.33, 'utility': 16784}
 7: {'name': '7', 'exp_sup': 1535.24, 'utility': 50790}
 8: {'name': '36', 'exp_sup': 1530.49, 'utility': 16934}
 9: {'name': '56', 'exp_sup': 1513.93, 'utility': 66780}
 10: {'name': '48', 'exp_sup': 1500.86, 'utility': 115367}
 11: {'name': '66', 'exp_sup': 1498.92, 'utility': 66644}
 12: {'name': '34', 'exp_sup': 1493.64, 'utility': 99330}
 13: {'name': '5', 'exp_sup': 1466.73, 'utility': 48369}
 14: {'name': '9', 'exp_sup': 1446.46, 'utility': 31818}
 15: {'name': '25', 'exp_sup': 1445.46, 'utility': 15465}
 16: {'name': '3', 'exp_sup': 1407.44, 'utility'

In [ ]:
# chạy với chess
algorithm = AlgorithmSKYFUP()
algorithm.run("./input_chess.txt, ./chess_utility.txt", 0.001, 500)
algorithm.print_stats("../out/SKYFUP/chess/output_chess_top_500.txt")

Reading data . . .
Minimum utility set to: 2156
{'database util: ': 2156659}
Top 500 CUPs:
 1: {'name': '52', 'exp_sup': 1616.54, 'utility': 104178}
 2: {'name': '29', 'exp_sup': 1598.92, 'utility': 70088}
 3: {'name': '58', 'exp_sup': 1595.58, 'utility': 17462}
 4: {'name': '40', 'exp_sup': 1579.27, 'utility': 188760}
 5: {'name': '60', 'exp_sup': 1565.64, 'utility': 68908}
 6: {'name': '62', 'exp_sup': 1537.33, 'utility': 16784}
 7: {'name': '7', 'exp_sup': 1535.24, 'utility': 50790}
 8: {'name': '36', 'exp_sup': 1530.49, 'utility': 16934}
 9: {'name': '56', 'exp_sup': 1513.93, 'utility': 66780}
 10: {'name': '48', 'exp_sup': 1500.86, 'utility': 115367}
 11: {'name': '66', 'exp_sup': 1498.92, 'utility': 66644}
 12: {'name': '34', 'exp_sup': 1493.64, 'utility': 99330}
 13: {'name': '5', 'exp_sup': 1466.73, 'utility': 48369}
 14: {'name': '9', 'exp_sup': 1446.46, 'utility': 31818}
 15: {'name': '25', 'exp_sup': 1445.46, 'utility': 15465}
 16: {'name': '3', 'exp_sup': 1407.44, 'utility'

In [ ]:
# chạy với chess
algorithm = AlgorithmSKYFUP()
algorithm.run("./input_chess.txt, ./chess_utility.txt", 0.001, 700)
algorithm.print_stats("../out/SKYFUP/chess/output_chess_top_700.txt")

Reading data . . .
Minimum utility set to: 2156
{'database util: ': 2156659}
Top 700 CUPs:
 1: {'name': '52', 'exp_sup': 1616.54, 'utility': 104178}
 2: {'name': '29', 'exp_sup': 1598.92, 'utility': 70088}
 3: {'name': '58', 'exp_sup': 1595.58, 'utility': 17462}
 4: {'name': '40', 'exp_sup': 1579.27, 'utility': 188760}
 5: {'name': '60', 'exp_sup': 1565.64, 'utility': 68908}
 6: {'name': '62', 'exp_sup': 1537.33, 'utility': 16784}
 7: {'name': '7', 'exp_sup': 1535.24, 'utility': 50790}
 8: {'name': '36', 'exp_sup': 1530.49, 'utility': 16934}
 9: {'name': '56', 'exp_sup': 1513.93, 'utility': 66780}
 10: {'name': '48', 'exp_sup': 1500.86, 'utility': 115367}
 11: {'name': '66', 'exp_sup': 1498.92, 'utility': 66644}
 12: {'name': '34', 'exp_sup': 1493.64, 'utility': 99330}
 13: {'name': '5', 'exp_sup': 1466.73, 'utility': 48369}
 14: {'name': '9', 'exp_sup': 1446.46, 'utility': 31818}
 15: {'name': '25', 'exp_sup': 1445.46, 'utility': 15465}
 16: {'name': '3', 'exp_sup': 1407.44, 'utility'

In [ ]:
# chạy với chess
algorithm = AlgorithmSKYFUP()
algorithm.run("./input_chess.txt, ./chess_utility.txt", 0.001, 900)
algorithm.print_stats("../out/SKYFUP/chess/output_chess_top_900.txt")

Reading data . . .
Minimum utility set to: 2156
{'database util: ': 2156659}
Top 900 CUPs:
 1: {'name': '52', 'exp_sup': 1616.54, 'utility': 104178}
 2: {'name': '29', 'exp_sup': 1598.92, 'utility': 70088}
 3: {'name': '58', 'exp_sup': 1595.58, 'utility': 17462}
 4: {'name': '40', 'exp_sup': 1579.27, 'utility': 188760}
 5: {'name': '60', 'exp_sup': 1565.64, 'utility': 68908}
 6: {'name': '62', 'exp_sup': 1537.33, 'utility': 16784}
 7: {'name': '7', 'exp_sup': 1535.24, 'utility': 50790}
 8: {'name': '36', 'exp_sup': 1530.49, 'utility': 16934}
 9: {'name': '56', 'exp_sup': 1513.93, 'utility': 66780}
 10: {'name': '48', 'exp_sup': 1500.86, 'utility': 115367}
 11: {'name': '66', 'exp_sup': 1498.92, 'utility': 66644}
 12: {'name': '34', 'exp_sup': 1493.64, 'utility': 99330}
 13: {'name': '5', 'exp_sup': 1466.73, 'utility': 48369}
 14: {'name': '9', 'exp_sup': 1446.46, 'utility': 31818}
 15: {'name': '25', 'exp_sup': 1445.46, 'utility': 15465}
 16: {'name': '3', 'exp_sup': 1407.44, 'utility'

## Hiện thực retail

In [ ]:
# chạy với retail
algorithm = AlgorithmSKYFUP()
algorithm.run("./input_retail.txt, ./retail_utility.txt", 0.0001, 100)
algorithm.print_stats("../out/SKYFUP/retail/output_retail_top_100.txt")

Reading data . . .
Minimum utility set to: 1491
{'database util: ': 14910915}
Top 100 CUPs:
 1: {'name': '40', 'exp_sup': 25268.8, 'utility': 277044}
 2: {'name': '49', 'exp_sup': 21135.04, 'utility': 461182}
 3: {'name': '39', 'exp_sup': 7775.43, 'utility': 169166}
 4: {'name': '33', 'exp_sup': 7519.53, 'utility': 250500}
 5: {'name': '42', 'exp_sup': 7426.48, 'utility': 245544}
 6: {'name': '40, 49', 'exp_sup': 7297.73, 'utility': 476144}
 7: {'name': '40, 42', 'exp_sup': 2824.5, 'utility': 247864}
 8: {'name': '40, 39', 'exp_sup': 2602.65, 'utility': 168244}
 9: {'name': '49, 42', 'exp_sup': 2261.59, 'utility': 245579}
 10: {'name': '66', 'exp_sup': 2237.39, 'utility': 49622}
 11: {'name': '40, 33', 'exp_sup': 2084.63, 'utility': 184488}
 12: {'name': '49, 33', 'exp_sup': 1998.65, 'utility': 219766}
 13: {'name': '49, 39', 'exp_sup': 1978.64, 'utility': 173210}
 14: {'name': '90', 'exp_sup': 1913.99, 'utility': 21008}
 15: {'name': '226', 'exp_sup': 1626.78, 'utility': 17780}
 16: {

In [ ]:
# chạy với retail
algorithm = AlgorithmSKYFUP()
algorithm.run("./input_retail.txt, ./retail_utility.txt", 0.0001, 300)
algorithm.print_stats("../out/SKYFUP/retail/output_retail_top_300.txt")

Reading data . . .
Minimum utility set to: 1491
{'database util: ': 14910915}
Top 300 CUPs:
 1: {'name': '40', 'exp_sup': 25268.8, 'utility': 277044}
 2: {'name': '49', 'exp_sup': 21135.04, 'utility': 461182}
 3: {'name': '39', 'exp_sup': 7775.43, 'utility': 169166}
 4: {'name': '33', 'exp_sup': 7519.53, 'utility': 250500}
 5: {'name': '42', 'exp_sup': 7426.48, 'utility': 245544}
 6: {'name': '40, 49', 'exp_sup': 7297.73, 'utility': 476144}
 7: {'name': '40, 42', 'exp_sup': 2824.5, 'utility': 247864}
 8: {'name': '40, 39', 'exp_sup': 2602.65, 'utility': 168244}
 9: {'name': '49, 42', 'exp_sup': 2261.59, 'utility': 245579}
 10: {'name': '66', 'exp_sup': 2237.39, 'utility': 49622}
 11: {'name': '40, 33', 'exp_sup': 2084.63, 'utility': 184488}
 12: {'name': '49, 33', 'exp_sup': 1998.65, 'utility': 219766}
 13: {'name': '49, 39', 'exp_sup': 1978.64, 'utility': 173210}
 14: {'name': '90', 'exp_sup': 1913.99, 'utility': 21008}
 15: {'name': '226', 'exp_sup': 1626.78, 'utility': 17780}
 16: {

In [ ]:
# chạy với retail
algorithm = AlgorithmSKYFUP()
algorithm.run("./input_retail.txt, ./retail_utility.txt", 0.0001, 500)
algorithm.print_stats("../out/SKYFUP/retail/output_retail_top_500.txt")

Reading data . . .
Minimum utility set to: 1491
{'database util: ': 14910915}
Top 500 CUPs:
 1: {'name': '40', 'exp_sup': 25268.8, 'utility': 277044}
 2: {'name': '49', 'exp_sup': 21135.04, 'utility': 461182}
 3: {'name': '39', 'exp_sup': 7775.43, 'utility': 169166}
 4: {'name': '33', 'exp_sup': 7519.53, 'utility': 250500}
 5: {'name': '42', 'exp_sup': 7426.48, 'utility': 245544}
 6: {'name': '40, 49', 'exp_sup': 7297.73, 'utility': 476144}
 7: {'name': '40, 42', 'exp_sup': 2824.5, 'utility': 247864}
 8: {'name': '40, 39', 'exp_sup': 2602.65, 'utility': 168244}
 9: {'name': '49, 42', 'exp_sup': 2261.59, 'utility': 245579}
 10: {'name': '66', 'exp_sup': 2237.39, 'utility': 49622}
 11: {'name': '40, 33', 'exp_sup': 2084.63, 'utility': 184488}
 12: {'name': '49, 33', 'exp_sup': 1998.65, 'utility': 219766}
 13: {'name': '49, 39', 'exp_sup': 1978.64, 'utility': 173210}
 14: {'name': '90', 'exp_sup': 1913.99, 'utility': 21008}
 15: {'name': '226', 'exp_sup': 1626.78, 'utility': 17780}
 16: {

In [ ]:
# chạy với retail
algorithm = AlgorithmSKYFUP()
algorithm.run("./input_retail.txt, ./retail_utility.txt", 0.0001, 700)
algorithm.print_stats("../out/SKYFUP/retail/output_retail_top_700.txt")

Reading data . . .
Minimum utility set to: 1491
{'database util: ': 14910915}
Top 700 CUPs:
 1: {'name': '40', 'exp_sup': 25268.8, 'utility': 277044}
 2: {'name': '49', 'exp_sup': 21135.04, 'utility': 461182}
 3: {'name': '39', 'exp_sup': 7775.43, 'utility': 169166}
 4: {'name': '33', 'exp_sup': 7519.53, 'utility': 250500}
 5: {'name': '42', 'exp_sup': 7426.48, 'utility': 245544}
 6: {'name': '40, 49', 'exp_sup': 7297.73, 'utility': 476144}
 7: {'name': '40, 42', 'exp_sup': 2824.5, 'utility': 247864}
 8: {'name': '40, 39', 'exp_sup': 2602.65, 'utility': 168244}
 9: {'name': '49, 42', 'exp_sup': 2261.59, 'utility': 245579}
 10: {'name': '66', 'exp_sup': 2237.39, 'utility': 49622}
 11: {'name': '40, 33', 'exp_sup': 2084.63, 'utility': 184488}
 12: {'name': '49, 33', 'exp_sup': 1998.65, 'utility': 219766}
 13: {'name': '49, 39', 'exp_sup': 1978.64, 'utility': 173210}
 14: {'name': '90', 'exp_sup': 1913.99, 'utility': 21008}
 15: {'name': '226', 'exp_sup': 1626.78, 'utility': 17780}
 16: {

In [ ]:
# chạy với retail
algorithm = AlgorithmSKYFUP()
algorithm.run("./input_retail.txt, ./retail_utility.txt", 0.0001, 900)
algorithm.print_stats("../out/SKYFUP/retail/output_retail_top_900.txt")

Reading data . . .
Minimum utility set to: 1491
{'database util: ': 14910915}
Top 900 CUPs:
 1: {'name': '40', 'exp_sup': 25268.8, 'utility': 277044}
 2: {'name': '49', 'exp_sup': 21135.04, 'utility': 461182}
 3: {'name': '39', 'exp_sup': 7775.43, 'utility': 169166}
 4: {'name': '33', 'exp_sup': 7519.53, 'utility': 250500}
 5: {'name': '42', 'exp_sup': 7426.48, 'utility': 245544}
 6: {'name': '40, 49', 'exp_sup': 7297.73, 'utility': 476144}
 7: {'name': '40, 42', 'exp_sup': 2824.5, 'utility': 247864}
 8: {'name': '40, 39', 'exp_sup': 2602.65, 'utility': 168244}
 9: {'name': '49, 42', 'exp_sup': 2261.59, 'utility': 245579}
 10: {'name': '66', 'exp_sup': 2237.39, 'utility': 49622}
 11: {'name': '40, 33', 'exp_sup': 2084.63, 'utility': 184488}
 12: {'name': '49, 33', 'exp_sup': 1998.65, 'utility': 219766}
 13: {'name': '49, 39', 'exp_sup': 1978.64, 'utility': 173210}
 14: {'name': '90', 'exp_sup': 1913.99, 'utility': 21008}
 15: {'name': '226', 'exp_sup': 1626.78, 'utility': 17780}
 16: {

# Thuật toán 2: UHUOPM

## Thư viện

In [ ]:
import sys, os
import time
import tracemalloc
import random

## Dữ liệu thử nghiệm

In [ ]:
# Example
# a b c d e f
# 0.9 0 0.9 0.6 0 0
# 0.9 0.9 0.7 0.6 0.4 0
# 0 0.5 0.8 0.9 0 0.2
# 0 0 0.9 0 0.1 0.5
# 0.4 0.5 0.9 0.3 0 0
# 0 0 0 0.9 0.1 0.6
# 0.9 0.7 0 0 0 0

# Example utility
# a c d:9:3 1 5
# a b c d e:20:6 2 3 5 4
# b c d f:17:2 6 5 4
# c e f:15:5 4 6
# a b c d:23:6 4 3 10
# d e f:13:5 4 4
# a b:5:3 2

## Cài đặt cấu trúc dữ liệu

### PUO-lists

In [ ]:
class PUO:
    """
    Là một đối tượng lưu trữ các thông tin i của mỗi giao dịch (TID, xác xuất, tiện ích, tiện ích còn lại, tổng tiện ích giao dịch, tiện ích của giao dịch khi tiện ích còn lại bằng không)
    """
    def __init__(self, TID: int, prob: float, utility: int, r_utility: int, twu: int) -> None:
        """
        Khởi tạo PUO

        Args:
            TID (int): mã giao dịch
            prob (float): xác xuất tồn tại
            utility (int): tiện ích giao dịch
            r_utility (int): tiện ích giao dịch còn lại
            twu (int): tổng tiện ích giao dịch
            zrutils (int): tiện ích giao dịch còn lại khi tiện ích còn lại bằng không
        """

        """
        Dựa vào dữ liệu thử nghiệm ta có

        Đối với item A và giao dịch 1

        TID = 1, pro = 0.9, utility = 3, r_utility = 6, tu = 9, zr_utility = 0
        """
        self.TID = TID
        self.prob = prob
        self.utility = utility
        self.r_utility = r_utility
        self.twu = twu
        self.zrutils = self.utility if self.r_utility == 0 else 0

    def __repr__(self):
        return f"PUO(TID={self.TID}, prob={self.prob:.4f}, utility={self.utility}, r_utility={self.r_utility}, twu={self.twu})\n"


In [ ]:
class PUOList:
    """
    Là một đối tượng lưu trữ danh sách giao dịch của item i (PUO)
    """
    def __init__(self, name=None) -> None:
        """
        Khởi tạo PUOList

        Args:
            name (str): tên của PUOList hoặc tên của item
            PUO (PUO): danh sách PUO của item
        """

        """
        Dựa vào dữ liệu thử nghiệm ta có

        Đối với item A:
            A
                TID     pro     utility     r_utility       twu     zrutils
                1       0.9     3           6               9       0
                2       0.9     6           14              20      0
                3       0.4     6           17              23      0
                4       0.9     3           2               5       0

        """

        self.name = name
        self.puo_list = []

    def add_puo(self, PUO) -> None:
        """
        Thêm PUO vào danh sách PUO

        Args:
            PUO (PUO): PUO của item cần thêm
        """

        self.puo_list.append(PUO)

    def __repr__(self):
        return f"PUOList(Name={self.name}, PUO={self.puo_list})\n"


In [ ]:
class PFUTable:
    """
    PFUTable là đối tượng chứa các thông tin của một i bao gồm: tên, số lần xuất hiện, giá trị tiện ích, giá trị tiện ích còn lại,
        tiện ích của giao dịch, giá trị xác suất, giá trị tiện ích khi tiện ích còn lại bằng không, danh sách PUO của item
    """
    def __init__(self, name=None, puo_list=None) -> None:
        """
        Khởi tạo PFUTable

        Args:
            name (str): tên của cup (tức tên của i)
            puo_list (PUOList): danh sách PUOList của i
        """
        self.name = name
        self.sup = len(puo_list.puo_list) if puo_list.puo_list else 0
        self.utility = round(sum(suo_list.utility for suo_list in puo_list.puo_list) if puo_list.puo_list else 0, 2)
        self.r_utility = round(sum(suo_list.r_utility for suo_list in puo_list.puo_list) if puo_list.puo_list else 0, 2)
        self.twu = sum(suo_list.twu for suo_list in puo_list.puo_list) if puo_list.puo_list else 0
        self.pro = sum(suo_list.prob for suo_list in puo_list.puo_list) if puo_list.puo_list else 0
        self.zrutils = round(sum(suo_list.utility for suo_list in puo_list.puo_list if suo_list.r_utility == 0) if puo_list.puo_list else 0, 2)

    def __repr__(self):
        return f"PFUTable(name={self.name}, sup={self.sup}, pro={self.pro:.4f}, utility={self.utility}, r_utility={self.r_utility}, twu={self.twu})\n"

## Cài đặt thuật toán

In [ ]:
class UHUOPM:
    """
    Triển khai thuật toán
    """
    def __init__(self):
        self.start_timestamp = 0
        self.end_timestamp = 0
        self.database_size = 0
        self.candidate = 0
        self.puo_list_dict = {}
        self.pfu_table_dict = {}
        self.item_name = []
        self.min_pro = 0
        self.min_utility = 0
        self.PHUOPs = set()
        self.peak_memory_usage = 0

    def read_data(self, file_path: str) -> None:
        """
        Đọc cơ sở dữ liệu

        Args:
            file_path (str): đường dẫn chứa file dữ liệu
        """
        file_paths = file_path.split(", ")
        try:
            with open(file_paths[0], 'r') as file1, open(file_paths[1], 'r') as file2:
                print("Reading data . . .")
                tlines = file1.readlines()
                ulines = file2.readlines()
                self.database_size = len(tlines)
                return tlines, ulines
        except FileNotFoundError as e:
            print(f"File not found: {e}")
            print("STOP ALGORITHM !!!")
            sys.exit(0)
        except Exception as e:
            print(f"An error occurred: {e}")
            sys.exit(0)

    def process_data(self, file_path: str) -> None:
        """
        Xử lý giao dịch và tạo PFUTable

        Args:
            file_path (str): đường dẫn chứa file dữ liệu
        """

        tlines, ulines = self.read_data(file_path)

        self.is_name = tlines[0].strip().split(" ")

        # duyệt và tạo PUOList cho từng item
        for i in self.is_name:
            self.puo_list_dict[i] = PUOList(i)

        # duyệt và tính toán các giá trị cho item
        for idx_line, line in enumerate(tlines[1:]):
            # xử lý chuỗi đầu vào
            probabilities = list(map(float, line.strip().split()))
            utility = ulines[idx_line]
            parts = utility.split(":")
            keys = parts[0].split()
            total_sum = int(parts[1])
            values = list(map(int, parts[2].split()))

            result = {key: value for key, value in zip(keys, values)}

            # tính toán các giá trị
            for idx, prob in enumerate(probabilities):
                if prob > 0:
                    i = self.is_name[idx]
                    if i in result.keys():
                        positions = keys.index(i)
                        utility = result[i]
                        r_utility = sum(values[positions+1:])
                        # thêm PUO vào danh sách chứa PUO của item tương ứng
                        self.puo_list_dict[i].add_puo(PUO(idx_line + 1, prob, utility, r_utility, total_sum))

        # tạo PFUTable tương ứng item
        for idx, i in enumerate(self.is_name):
            self.pfu_table_dict[i] = PFUTable(i, self.puo_list_dict.get(i))

    def run_UHUOPM(self, file_path: str, k: int) -> None:
        """
        Khởi chạy thuật toán

        Args:
            file_path (str): đường dẫn vào cơ sở dữ liệu
            k (int): số lượng mẫu cần khai thác
        """
        self.start_timestamp = time.time()

        # xử lý dữ liệu
        self.process_data(file_path)

        # lấy k phần tử có utility cao nhất
        pfu_utility = sorted(self.pfu_table_dict.values(), key=lambda x: x.utility, reverse=True)[:k]

        # sắp xếp lại các phần tử theo thứ tự twu giảm dần
        pfu_twu = sorted(pfu_utility, key=lambda x: x.twu)
        item_name = [i.name for i in pfu_twu]

        # tính toán độ hội tụ để xóa các phần tử không tạo tiệ ích cao trong tương lai
        r_item = self.coverage(item_name)

        # danh sách các item được xem xét
        self.item_name = [i for i in r_item]

        # khởi tạo ngưỡng tiện ích ban đầu
        min_utility = sorted(self.item_name, key=lambda x: self.pfu_table_dict.get(x).utility, reverse=True)[:k]
        self.min_utility = self.pfu_table_dict.get(min_utility[-1]).utility

        # khởi tạo ngưỡng xác suất ban đầu
        min_pro = sorted(self.item_name, key=lambda x: self.pfu_table_dict.get(x).pro, reverse=True)[:k]
        self.min_pro = self.pfu_table_dict.get(min_pro[-1]).pro

        # xét duyệt và kiểm tra các ứng viên để thêm vào danh sách PHUOPs
        for i in self.item_name:
            self.judge(i, k)

        # sắp xếp các ứng viên theo thứ tự giảm dần pro
        self.item_name = sorted(self.item_name, key=lambda x: self.pfu_table_dict.get(x).pro, reverse=True)[:k]

        # tính số lượng ứng viên
        self.candidate += len(self.item_name)

        # bắt đầu theo dõi bộ nhớ
        tracemalloc.start()

        # bắt đầu tìm kiếm
        self.PHUOP_Search("", self.item_name, k)
        self.PHUOPs = sorted(self.PHUOPs, key=lambda x: float(x[2].split(': ')[1]), reverse=True)[:k]

        # in kết quả
        i = 1
        for pfu in self.PHUOPs:
            print(f" {i}: {pfu}")
            i +=1

        # kết thúc theo dõi và lấy thông tin
        _, self.peak_memory_usage = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        self.peak_memory_usage /= 10**6 # chuyển đổi sang MB
        self.end_timestamp = time.time()


    def calculate_lu(self, x_a, x_b):
        """
        Tính toán local utility của hai item

        Args:
            x_a (str): tên item a
            x_b (str): tên item b

        Returns:
            lu (int): lu của item a và item b
        """
        lu = 0

        # lấy danh sách PUO của từng item
        i_a = self.puo_list_dict.get(x_a).puo_list
        i_b = self.puo_list_dict.get(x_b).puo_list
        i, j = 0, 0

        # duyệt qua từng phần của 2 danh sách PUO
        while i < len(i_a) and j < len(i_b):
            # nếu trùng TID
            if i_a[i].TID == i_b[j].TID:
                # tính tổng utility và r_utility của item a tại giao dịch đó
                  lu += (i_a[i].utility + i_a[i].r_utility)
                  i += 1
                  j += 1
            # nếu TID của item a nhỏ hơn TID của item b thì tăng giao dịch của item a
            elif i_a[i].TID < i_b[j].TID:
                i += 1
            # ngược lại
            else:
                j += 1

        return lu

    def calculate_twu(self, x_a: str, x_b: str):
        """
        Tính toán local utility của hai item

        Args:
            x_a (str): tên item a
            x_b (str): tên item b

        Returns:
            twu (int): twu của item a và item b
        """

        twu = 0
        i, j = 0, 0

         # lấy danh sách PUO của từng item
        i_a = self.puo_list_dict.get(x_a).puo_list
        i_b = self.puo_list_dict.get(x_b).puo_list

        # duyệt qua từng phần của 2 danh sách PUO
        while i < len(i_a) and j < len(i_b):
            # nếu trùng TID
            if i_a[i].TID == i_b[j].TID:
                # tính tổng twu của item a tại giao dịch đó
                twu += i_a[i].twu
                i += 1
                j += 1
            # nếu TID của item a nhỏ hơn TID của item b thì tăng giao dịch của item a
            elif i_a[i].TID < i_b[j].TID:
                i += 1
            # ngược lại
            else:
                j += 1
        return twu

    def coverage(self, item_list: str) -> str:
        """
        Tính toán độ hội tụ và xóa các phần tử hội tụ không tốt

        Args:
            item_list (str): danh sách item cần xét

        Returns:
            item_list (str): danh sách các item có độ hội tụ tốt
        """

        r_item = []
        # duyệt các item
        for idx, x in enumerate(item_list):
            # duyệt các item sau item trước
            for y in item_list[idx+1:]:
                # kiểm tra nếu twu của item x, y bằng twu của twu của x thì item x đó không hội tụ
                if self.calculate_twu(x, y) == self.pfu_table_dict.get(x).twu:
                    r_item.append(x)
                    break
        # xóa các item cần xóa cho item_list
        for i in r_item:
            item_list.remove(i)
        return item_list


    def PHUOP_Search(self, X: str, exten_of_x: list, k: int) -> None:
        """
        Khai thác mẫu từ danh sách ứng viên

        Args:
            X (str): tiền tố của item đang xét
            exten_of_x (list): danh sách ứng viên tham gia
            k (int): số lượng mẫu cần tìm
        """

        # nếu số lượng phần tử mở rộng nhỏ hơn 1 thì dừng
        if len(exten_of_x) <= 1:
            return

        # duyệt qua các ứng viên
        for idx, x_a in enumerate(exten_of_x):
            pfu_x_a = self.pfu_table_dict.get(x_a)

            # nếu xác suất nhỏ hơn ngưỡng xác suất thì dừng lại
            if pfu_x_a.pro < self.min_pro:
                return

            x_u_ru = pfu_x_a.utility + pfu_x_a.r_utility
            # kiểm tra độ mở rộng của item đang xét có lớn hơn ngưỡng tiện ích
            if x_u_ru - pfu_x_a.zrutils > self.min_utility:
                # kiểm tra điều kiện xác suất của item
                if pfu_x_a.pro > self.min_pro:
                    exten = []

                    # duyệt các phần tử cần kết hợp
                    for x_b in exten_of_x[idx + 1:]:

                        # kiểm tra điều kiện xác suất và local utility của item đang xét và item cần mở rộng
                        if self.pfu_table_dict.get(x_b).pro < self.min_pro or self.calculate_lu(x_a, x_b) < self.min_utility:
                            break

                        puo_list = self.puo_list_dict.get(x_b)
                        max_prob = max(puo.prob for puo in puo_list.puo_list)
                        pfu_x_b = self.pfu_table_dict.get(x_b)
                        max_util = max(puo.utility for puo in puo_list.puo_list)

                        # kiểm tra ngưỡng tối đa và ngưỡng tiện ích mở rộng
                        if pfu_x_a.pro * max_prob < self.min_pro or pfu_x_a.utility + max_util < self.min_utility:
                            break

                        # kiểm tra 'du'
                        # nếu thỏa điều kiện thì kết hợp item đang xét với item cần kết hợp
                        du = pfu_x_b.r_utility + pfu_x_b.utility + (self.pfu_table_dict.get(X).utility if X in self.pfu_table_dict else 0)
                        if du > self.min_utility:
                            if len(x_b) < len(X):
                                x_ab = self.construct('', x_a, x_b)
                            else:
                                x_ab = self.construct(X, x_a, x_b)

                            # nếu item kết hợp thỏa thì kiểm tra
                            if x_ab != None:
                                pfu_x_ab = self.pfu_table_dict.get(x_ab)
                                # kiểm tra các ngưỡng tiện ích, xác suất và thêm vào PHOUPs
                                if pfu_x_ab.utility >= self.min_utility :
                                    if pfu_x_ab.pro > self.min_pro:
                                        self.judge(x_ab, k)

                                # kiểm tra khả năng mở rộng của item mở rộng
                                if pfu_x_ab.pro > self.min_pro and pfu_x_ab.r_utility + pfu_x_ab.utility > self.min_utility:
                                    exten.append(x_ab)

                        # nếu du không thỏa thì kiểm tra khả năng mở rộng của item cần kết hợp
                        elif pfu_x_b.r_utility + pfu_x_b.utility > self.min_utility and pfu_x_b.pro > self.min_pro:
                            # thỏa thì thêm item cần mở rộng
                            exten.append(x_b)

                    # nâng ngưỡng xác suất them item có xác suất nhỏ nhất của danh sách mở rộng
                    if exten:
                        min_pro_item = min(exten, key=lambda x: self.pfu_table_dict.get(x).pro)
                        min_pro_value = self.pfu_table_dict.get(min_pro_item).pro
                        self.min_pro = min_pro_value

                    # tính số lượng ứng cử viên
                    self.candidate += len(exten)
                    self.PHUOP_Search(x_a, exten, k)

    def judge(self, x: str, k: int) -> None:
        """
        Thêm phần tử vào PHOUPs và cập nhật lại ngưỡng tiện ích, xác suất

        Args:
            x_a (str): tên của item cần thêm
            k (int): số lượng mẫu khai thác
        """

        pfu_x = self.pfu_table_dict.get(x)

        # thêm phần tử mới theo format: Name, pro, utility
        # Name: a, c, d: prob: 0.97: utility: 42

        self.PHUOPs.add((
            f"Name: {x}",
            f"prob: {pfu_x.pro:.2f}",
            f"utility: {pfu_x.utility}"
        ))

        # nếu số lượng phần tử PHUOPs lớn hơn k
        if len(self.PHUOPs) > k:
            # xóa phần tử có giá trị xác suất nhỏ nhất
            min_pro_item = min(
                self.PHUOPs,
                key=lambda x: float(x[1].split(': ')[1])
             )
            self.PHUOPs.remove(min_pro_item)

            # xóa phần tử có giá trị tiện ích nhỏ nhất
            min_utility_item = min(
                self.PHUOPs,
                key=lambda x: float(x[2].split(': ')[1])
             )
            self.PHUOPs.remove(min_utility_item)

            # cập nhật lại ngưỡng xác suất
            self.min_pro = float(min(
                self.PHUOPs,
                key=lambda x: float(x[1].split(': ')[1])
            )[1].split(': ')[1])

            # cập nhật lại ngưỡng tiện ích
            self.min_utility = int(min(
                self.PHUOPs,
                key=lambda x: float(x[2].split(': ')[1])
            )[2].split(': ')[1])

    def construct(self, x: str, xa: str, xb: str) -> None:
        """
        Kết hợp hai item a và b

        Args:
            x (str): tiền tố
            xa (str): tên của item a
            xb (str): tên của item b
        """

        # tạo tên cho item kết hợp a và b
        xa_list = xa.split(", ")
        xb_list = xb.split(", ")
        unique_numbers = set(xa_list + xb_list)
        x_ab = ", ".join(sorted(unique_numbers))

        # tạo danh sách PUO của item ab
        self.puo_list_dict[x_ab] = PUOList(x_ab)

        # lấy danh sách PUO của item a và b
        Ea = self.puo_list_dict[xa].puo_list
        Eb = self.puo_list_dict[xb].puo_list
        i, j = 0, 0

        # tính điều kiện t
        pa, pb = self.pfu_table_dict.get(xa), self.pfu_table_dict.get(xb)
        t = pa.utility + pa.r_utility + pb.utility + pb.r_utility
        flag = False

        # xét duyệt các phần tử trong danh sách PUO của phần tử a và b
        while i < len(Ea) and j < len(Eb):
            # nếu TID của a trùng TID của b
            if Ea[i].TID == Eb[j].TID:
                flag = True
                # tính toán các các giá trị PUO cho item ab
                E_ab = PUO(Ea[i].TID, (Ea[i].prob * Eb[j].prob), Ea[i].utility + Eb[j].utility, Eb[j].r_utility, Ea[i].twu)
                self.puo_list_dict[x_ab].add_puo(E_ab)
                i += 1
                j += 1
            # nếu TID của item a nhỏ hơn TID của item b thì tăng giao dịch của item a
            elif Ea[i].TID < Eb[j].TID:
                # giảm t theo utility và r_utility của item a
                t = t - Ea[i].utility - Ea[i].r_utility
                i += 1
            # ngược lại
            else:
                # giảm t theo utility và r_utility của item b
                t = t - Eb[j].utility - Eb[j].r_utility
                j += 1

            # kiểm tra t có nhỏ hơn ngưỡng tiệc ích không
            if t < self.min_utility:
                # xóa phần item ab khỏi danh sách PUO
                self.puo_list_dict.pop(x_ab)
                return None

        # kiểm tra phần tử tiền tố
        if flag and x != '':
            i, j = 0, 0
            Eab = self.puo_list_dict[x_ab].puo_list
            E = self.puo_list_dict[x].puo_list

            # duyệt danh sách PUO của item ab và tiền tố
            while i < len(Eab) and j < len(E):
                # nếu trùng TID
                if Eab[i].TID == E[j].TID:
                    # cập nhật utility
                    Eab[i].utility -= E[j].utility
                    # cập nhật pro
                    Eab[i].prob = Eab[i].prob / E[j].prob
                    i+=1
                j += 1
        # tạo PFUTable của item ab
        self.pfu_table_dict[x_ab] = PFUTable(x_ab, self.puo_list_dict[x_ab])

        return x_ab

    def print_stats(self, path: str) -> None:
        """
        Thống kê thông tin kết quả chạy thuật toán

        Args:
            path (str): đường dẫn lưu thông tin
        """
        total_utility = 0
        directory = os.path.dirname(path)
        if directory and not os.path.exists(directory):
            os.makedirs(directory)

        with open(path, 'w') as output_file:
            output_file.write(f"minUtil: {self.min_utility}\n")
            for t in self.PHUOPs:
                total_utility += int(t[2].split(": ")[1])
                output_file.write(f"{t[0]}: {t[1]}: {t[2]}\n")
            output_file.write("=============  TOP-K UHOUPM - STATS =============\n")
            output_file.write(f" Transactions count from database : {self.database_size}\n")
            output_file.write(f" Candidates count : {self.candidate}\n")
            output_file.write(f" Sum utility : {total_utility}\n")
            output_file.write(f" Algorithm run time : {self.end_timestamp - self.start_timestamp:.0f} seconds\n")
            output_file.write(f" Peak memory usage : {self.peak_memory_usage:.2f} MB\n")


## Dữ liệu thực nghiệm

In [ ]:
# Chạy với Example
algorithm = UHUOPM()
algorithm.run_UHUOPM("./example.txt, ./example_utility.txt", 6)
algorithm.print_stats("../out/UHUOPM/output_example.txt")

Reading data . . .
 1: ('Name: c, d', 'prob: 1.95', 'utility: 38')
 2: ('Name: d', 'prob: 3.30', 'utility: 30')
 3: ('Name: a, c', 'prob: 1.80', 'utility: 22')
 4: ('Name: b, c', 'prob: 1.48', 'utility: 20')
 5: ('Name: c', 'prob: 4.20', 'utility: 18')


## Hiện thực với foodmart

In [ ]:
# chạy với foodmart
algorithm = UHUOPM()
algorithm.run_UHUOPM("./input_foodmart.txt, ./foodmart_utility.txt", 100)
algorithm.print_stats("../out/UHUOPM/foodmart/output_foodmart_top_100.txt")

Reading data . . .
 1: ('Name: 1373', 'prob: 13.55', 'utility: 25560')
 2: ('Name: 1292', 'prob: 12.60', 'utility: 24642')
 3: ('Name: 988', 'prob: 9.30', 'utility: 24187')
 4: ('Name: 225', 'prob: 8.04', 'utility: 22951')
 5: ('Name: 222', 'prob: 9.33', 'utility: 22910')
 6: ('Name: 272', 'prob: 13.54', 'utility: 22678')
 7: ('Name: 1426', 'prob: 9.96', 'utility: 21692')
 8: ('Name: 1161', 'prob: 10.54', 'utility: 21540')
 9: ('Name: 1045', 'prob: 9.48', 'utility: 21090')
 10: ('Name: 446', 'prob: 6.93', 'utility: 20625')
 11: ('Name: 83', 'prob: 6.68', 'utility: 20563')
 12: ('Name: 1518', 'prob: 7.94', 'utility: 20416')
 13: ('Name: 1180', 'prob: 8.06', 'utility: 19890')
 14: ('Name: 1012', 'prob: 11.65', 'utility: 19861')
 15: ('Name: 175', 'prob: 9.14', 'utility: 19635')
 16: ('Name: 1380', 'prob: 7.99', 'utility: 19548')
 17: ('Name: 1368', 'prob: 7.57', 'utility: 19525')
 18: ('Name: 1497', 'prob: 7.19', 'utility: 19406')
 19: ('Name: 937', 'prob: 7.24', 'utility: 19133')
 20: (

In [ ]:
# chạy với foodmart
algorithm = UHUOPM()
algorithm.run_UHUOPM("./input_foodmart.txt, ./foodmart_utility.txt", 300)
algorithm.print_stats("../out/UHUOPM/foodmart/output_foodmart_top_300.txt")

Reading data . . .
 1: ('Name: 1373', 'prob: 13.55', 'utility: 25560')
 2: ('Name: 1292', 'prob: 12.60', 'utility: 24642')
 3: ('Name: 988', 'prob: 9.30', 'utility: 24187')
 4: ('Name: 225', 'prob: 8.04', 'utility: 22951')
 5: ('Name: 222', 'prob: 9.33', 'utility: 22910')
 6: ('Name: 272', 'prob: 13.54', 'utility: 22678')
 7: ('Name: 1426', 'prob: 9.96', 'utility: 21692')
 8: ('Name: 1161', 'prob: 10.54', 'utility: 21540')
 9: ('Name: 1045', 'prob: 9.48', 'utility: 21090')
 10: ('Name: 446', 'prob: 6.93', 'utility: 20625')
 11: ('Name: 83', 'prob: 6.68', 'utility: 20563')
 12: ('Name: 1518', 'prob: 7.94', 'utility: 20416')
 13: ('Name: 1180', 'prob: 8.06', 'utility: 19890')
 14: ('Name: 1012', 'prob: 11.65', 'utility: 19861')
 15: ('Name: 175', 'prob: 9.14', 'utility: 19635')
 16: ('Name: 1380', 'prob: 7.99', 'utility: 19548')
 17: ('Name: 1368', 'prob: 7.57', 'utility: 19525')
 18: ('Name: 1497', 'prob: 7.19', 'utility: 19406')
 19: ('Name: 937', 'prob: 7.24', 'utility: 19133')
 20: (

In [ ]:
# chạy với foodmart
algorithm = UHUOPM()
algorithm.run_UHUOPM("./input_foodmart.txt, ./foodmart_utility.txt", 500)
algorithm.print_stats("../out/UHUOPM/foodmart/output_foodmart_top_500.txt")

Reading data . . .
 1: ('Name: 1373', 'prob: 13.55', 'utility: 25560')
 2: ('Name: 1292', 'prob: 12.60', 'utility: 24642')
 3: ('Name: 988', 'prob: 9.30', 'utility: 24187')
 4: ('Name: 225', 'prob: 8.04', 'utility: 22951')
 5: ('Name: 222', 'prob: 9.33', 'utility: 22910')
 6: ('Name: 272', 'prob: 13.54', 'utility: 22678')
 7: ('Name: 1426', 'prob: 9.96', 'utility: 21692')
 8: ('Name: 1161', 'prob: 10.54', 'utility: 21540')
 9: ('Name: 1045', 'prob: 9.48', 'utility: 21090')
 10: ('Name: 446', 'prob: 6.93', 'utility: 20625')
 11: ('Name: 83', 'prob: 6.68', 'utility: 20563')
 12: ('Name: 1518', 'prob: 7.94', 'utility: 20416')
 13: ('Name: 1180', 'prob: 8.06', 'utility: 19890')
 14: ('Name: 1012', 'prob: 11.65', 'utility: 19861')
 15: ('Name: 175', 'prob: 9.14', 'utility: 19635')
 16: ('Name: 1380', 'prob: 7.99', 'utility: 19548')
 17: ('Name: 1368', 'prob: 7.57', 'utility: 19525')
 18: ('Name: 1497', 'prob: 7.19', 'utility: 19406')
 19: ('Name: 937', 'prob: 7.24', 'utility: 19133')
 20: (

In [ ]:
# chạy với foodmart
algorithm = UHUOPM()
algorithm.run_UHUOPM("./input_foodmart.txt, ./foodmart_utility.txt", 700)
algorithm.print_stats("../out/UHUOPM/foodmart/output_foodmart_top_700.txt")

Reading data . . .
 1: ('Name: 1373', 'prob: 13.55', 'utility: 25560')
 2: ('Name: 1292', 'prob: 12.60', 'utility: 24642')
 3: ('Name: 988', 'prob: 9.30', 'utility: 24187')
 4: ('Name: 225', 'prob: 8.04', 'utility: 22951')
 5: ('Name: 222', 'prob: 9.33', 'utility: 22910')
 6: ('Name: 272', 'prob: 13.54', 'utility: 22678')
 7: ('Name: 1426', 'prob: 9.96', 'utility: 21692')
 8: ('Name: 1161', 'prob: 10.54', 'utility: 21540')
 9: ('Name: 1045', 'prob: 9.48', 'utility: 21090')
 10: ('Name: 446', 'prob: 6.93', 'utility: 20625')
 11: ('Name: 83', 'prob: 6.68', 'utility: 20563')
 12: ('Name: 1518', 'prob: 7.94', 'utility: 20416')
 13: ('Name: 1180', 'prob: 8.06', 'utility: 19890')
 14: ('Name: 1012', 'prob: 11.65', 'utility: 19861')
 15: ('Name: 175', 'prob: 9.14', 'utility: 19635')
 16: ('Name: 1380', 'prob: 7.99', 'utility: 19548')
 17: ('Name: 1368', 'prob: 7.57', 'utility: 19525')
 18: ('Name: 1497', 'prob: 7.19', 'utility: 19406')
 19: ('Name: 937', 'prob: 7.24', 'utility: 19133')
 20: (

In [ ]:
# chạy với foodmart
algorithm = UHUOPM()
algorithm.run_UHUOPM("./input_foodmart.txt, ./foodmart_utility.txt", 900)
algorithm.print_stats("../out/UHUOPM/foodmart/output_foodmart_top_900.txt")

Reading data . . .
 1: ('Name: 1373', 'prob: 13.55', 'utility: 25560')
 2: ('Name: 1292', 'prob: 12.60', 'utility: 24642')
 3: ('Name: 988', 'prob: 9.30', 'utility: 24187')
 4: ('Name: 225', 'prob: 8.04', 'utility: 22951')
 5: ('Name: 222', 'prob: 9.33', 'utility: 22910')
 6: ('Name: 272', 'prob: 13.54', 'utility: 22678')
 7: ('Name: 1426', 'prob: 9.96', 'utility: 21692')
 8: ('Name: 1161', 'prob: 10.54', 'utility: 21540')
 9: ('Name: 1045', 'prob: 9.48', 'utility: 21090')
 10: ('Name: 446', 'prob: 6.93', 'utility: 20625')
 11: ('Name: 83', 'prob: 6.68', 'utility: 20563')
 12: ('Name: 1518', 'prob: 7.94', 'utility: 20416')
 13: ('Name: 1180', 'prob: 8.06', 'utility: 19890')
 14: ('Name: 1012', 'prob: 11.65', 'utility: 19861')
 15: ('Name: 175', 'prob: 9.14', 'utility: 19635')
 16: ('Name: 1380', 'prob: 7.99', 'utility: 19548')
 17: ('Name: 1368', 'prob: 7.57', 'utility: 19525')
 18: ('Name: 1497', 'prob: 7.19', 'utility: 19406')
 19: ('Name: 937', 'prob: 7.24', 'utility: 19133')
 20: (

## Hiện thực Chess

In [ ]:
# chạy với chess
algorithm = UHUOPM()
algorithm.run_UHUOPM("./input_chess.txt, ./chess_utility.txt", 100)
algorithm.print_stats("../out/UHUOPM/chess/output_chess_top_100.txt")

Reading data . . .
 1: ('Name: 40, 52', 'prob: 800.31', 'utility: 290726')
 2: ('Name: 40, 48', 'prob: 741.82', 'utility: 290391')
 3: ('Name: 34, 40', 'prob: 737.94', 'utility: 277239')
 4: ('Name: 29, 40', 'prob: 792.35', 'utility: 255631')
 5: ('Name: 40, 60', 'prob: 773.62', 'utility: 253428')
 6: ('Name: 40, 66', 'prob: 734.99', 'utility: 243649')
 7: ('Name: 40, 56', 'prob: 746.84', 'utility: 243050')
 8: ('Name: 40, 7', 'prob: 757.98', 'utility: 231271')
 9: ('Name: 3, 40', 'prob: 693.79', 'utility: 226522')
 10: ('Name: 40, 5', 'prob: 717.76', 'utility: 223282')
 11: ('Name: 40, 72', 'prob: 582.53', 'utility: 214715')
 12: ('Name: 48, 52', 'prob: 761.59', 'utility: 212460')
 13: ('Name: 40, 64', 'prob: 671.08', 'utility: 212460')
 14: ('Name: 34, 48', 'prob: 699.50', 'utility: 201383')
 15: ('Name: 40, 9', 'prob: 704.22', 'utility: 200832')
 16: ('Name: 36, 40', 'prob: 761.37', 'utility: 198648')
 17: ('Name: 34, 52', 'prob: 750.72', 'utility: 197472')
 18: ('Name: 40, 62', 'pr

In [ ]:
# chạy với chess
algorithm = UHUOPM()
algorithm.run_UHUOPM("./input_chess.txt, ./chess_utility.txt", 100)
algorithm.print_stats("../out/UHUOPM/chess/output_chess_top_100.txt")

Reading data . . .
 1: ('Name: 40, 52', 'prob: 800.31', 'utility: 290726')
 2: ('Name: 40, 48', 'prob: 741.82', 'utility: 290391')
 3: ('Name: 34, 40', 'prob: 737.94', 'utility: 277239')
 4: ('Name: 29, 40', 'prob: 792.35', 'utility: 255631')
 5: ('Name: 40, 60', 'prob: 773.62', 'utility: 253428')
 6: ('Name: 40, 66', 'prob: 734.99', 'utility: 243649')
 7: ('Name: 40, 56', 'prob: 746.84', 'utility: 243050')
 8: ('Name: 40, 7', 'prob: 757.98', 'utility: 231271')
 9: ('Name: 3, 40', 'prob: 693.79', 'utility: 226522')
 10: ('Name: 40, 5', 'prob: 717.76', 'utility: 223282')
 11: ('Name: 40, 72', 'prob: 582.53', 'utility: 214715')
 12: ('Name: 48, 52', 'prob: 761.59', 'utility: 212460')
 13: ('Name: 40, 64', 'prob: 671.08', 'utility: 212460')
 14: ('Name: 34, 48', 'prob: 699.50', 'utility: 201383')
 15: ('Name: 40, 9', 'prob: 704.22', 'utility: 200832')
 16: ('Name: 36, 40', 'prob: 761.37', 'utility: 198648')
 17: ('Name: 34, 52', 'prob: 750.72', 'utility: 197472')
 18: ('Name: 40, 62', 'pr

In [ ]:
# chạy với chess
algorithm = UHUOPM()
algorithm.run_UHUOPM("./input_chess.txt, ./chess_utility.txt", 300)
algorithm.print_stats("../out/UHUOPM/chess/output_chess_top_300.txt")

Reading data . . .
 1: ('Name: 40, 48, 52', 'prob: 378.43', 'utility: 385381')
 2: ('Name: 34, 40, 52', 'prob: 373.71', 'utility: 373506')
 3: ('Name: 34, 40, 48', 'prob: 347.15', 'utility: 366010')
 4: ('Name: 29, 40, 52', 'prob: 398.42', 'utility: 356097')
 5: ('Name: 40, 52, 60', 'prob: 394.11', 'utility: 352926')
 6: ('Name: 29, 40, 48', 'prob: 366.08', 'utility: 351549')
 7: ('Name: 40, 48, 60', 'prob: 366.75', 'utility: 351142')
 8: ('Name: 29, 34, 40', 'prob: 368.79', 'utility: 340850')
 9: ('Name: 40, 52, 66', 'prob: 373.34', 'utility: 339395')
 10: ('Name: 40, 52, 56', 'prob: 382.16', 'utility: 339206')
 11: ('Name: 40, 48, 66', 'prob: 350.19', 'utility: 338147')
 12: ('Name: 34, 40, 60', 'prob: 363.90', 'utility: 336770')
 13: ('Name: 40, 48, 56', 'prob: 343.40', 'utility: 332384')
 14: ('Name: 40, 52, 7', 'prob: 382.94', 'utility: 328740')
 15: ('Name: 34, 40, 66', 'prob: 343.62', 'utility: 325229')
 16: ('Name: 40, 48, 7', 'prob: 357.88', 'utility: 325169')
 17: ('Name: 34,

In [ ]:
# chạy với chess
algorithm = UHUOPM()
algorithm.run_UHUOPM("./input_chess.txt, ./chess_utility.txt", 500)
algorithm.print_stats("../out/UHUOPM/chess/output_chess_top_500.txt")

Reading data . . .
 1: ('Name: 40, 48, 52', 'prob: 378.43', 'utility: 385381')
 2: ('Name: 34, 40, 52', 'prob: 373.71', 'utility: 373506')
 3: ('Name: 29, 40, 52', 'prob: 398.42', 'utility: 356097')
 4: ('Name: 40, 52, 60', 'prob: 394.11', 'utility: 352926')
 5: ('Name: 40, 52, 66', 'prob: 373.34', 'utility: 339395')
 6: ('Name: 40, 52, 56', 'prob: 382.16', 'utility: 339206')
 7: ('Name: 40, 52, 7', 'prob: 382.94', 'utility: 328740')
 8: ('Name: 29, 40, 60', 'prob: 384.53', 'utility: 318464')
 9: ('Name: 40, 5, 52', 'prob: 368.61', 'utility: 317307')
 10: ('Name: 3, 40, 52', 'prob: 355.63', 'utility: 316103')
 11: ('Name: 40, 52, 58', 'prob: 401.34', 'utility: 306482')
 12: ('Name: 40, 52, 64', 'prob: 344.17', 'utility: 297061')
 13: ('Name: 36, 40, 52', 'prob: 382.23', 'utility: 296932')
 14: ('Name: 29, 40, 7', 'prob: 380.83', 'utility: 296027')
 15: ('Name: 40, 52, 62', 'prob: 391.25', 'utility: 294543')
 16: ('Name: 34, 48, 52', 'prob: 354.20', 'utility: 292525')
 17: ('Name: 40, 5

In [ ]:
# chạy với chess
algorithm = UHUOPM()
algorithm.run_UHUOPM("./input_chess.txt, ./chess_utility.txt", 700)
algorithm.print_stats("../out/UHUOPM/chess/output_chess_top_700.txt")

Reading data . . .
 1: ('Name: 40, 48, 52', 'prob: 378.43', 'utility: 385381')
 2: ('Name: 34, 40, 52', 'prob: 373.71', 'utility: 373506')
 3: ('Name: 29, 40, 52', 'prob: 398.42', 'utility: 356097')
 4: ('Name: 40, 52, 60', 'prob: 394.11', 'utility: 352926')
 5: ('Name: 40, 52, 66', 'prob: 373.34', 'utility: 339395')
 6: ('Name: 40, 52, 56', 'prob: 382.16', 'utility: 339206')
 7: ('Name: 40, 52, 7', 'prob: 382.94', 'utility: 328740')
 8: ('Name: 29, 40, 60', 'prob: 384.53', 'utility: 318464')
 9: ('Name: 40, 5, 52', 'prob: 368.61', 'utility: 317307')
 10: ('Name: 3, 40, 52', 'prob: 355.63', 'utility: 316103')
 11: ('Name: 40, 52, 58', 'prob: 401.34', 'utility: 306482')
 12: ('Name: 40, 52, 64', 'prob: 344.17', 'utility: 297061')
 13: ('Name: 36, 40, 52', 'prob: 382.23', 'utility: 296932')
 14: ('Name: 29, 40, 7', 'prob: 380.83', 'utility: 296027')
 15: ('Name: 40, 52, 62', 'prob: 391.25', 'utility: 294543')
 16: ('Name: 34, 48, 52', 'prob: 354.20', 'utility: 292525')
 17: ('Name: 40, 5

In [ ]:
# chạy với chess
algorithm = UHUOPM()
algorithm.run_UHUOPM("./input_chess.txt, ./chess_utility.txt", 900)
algorithm.print_stats("../out/UHUOPM/out/output_chess_top_900.txt")

Reading data . . .
 1: ('Name: 40, 48, 52', 'prob: 378.43', 'utility: 385381')
 2: ('Name: 34, 40, 52', 'prob: 373.71', 'utility: 373506')
 3: ('Name: 29, 40, 52', 'prob: 398.42', 'utility: 356097')
 4: ('Name: 40, 52, 60', 'prob: 394.11', 'utility: 352926')
 5: ('Name: 40, 52, 66', 'prob: 373.34', 'utility: 339395')
 6: ('Name: 40, 52, 56', 'prob: 382.16', 'utility: 339206')
 7: ('Name: 40, 52, 7', 'prob: 382.94', 'utility: 328740')
 8: ('Name: 29, 40, 60', 'prob: 384.53', 'utility: 318464')
 9: ('Name: 40, 5, 52', 'prob: 368.61', 'utility: 317307')
 10: ('Name: 3, 40, 52', 'prob: 355.63', 'utility: 316103')
 11: ('Name: 40, 52, 58', 'prob: 401.34', 'utility: 306482')
 12: ('Name: 40, 52, 64', 'prob: 344.17', 'utility: 297061')
 13: ('Name: 36, 40, 52', 'prob: 382.23', 'utility: 296932')
 14: ('Name: 29, 40, 7', 'prob: 380.83', 'utility: 296027')
 15: ('Name: 40, 52, 62', 'prob: 391.25', 'utility: 294543')
 16: ('Name: 34, 48, 52', 'prob: 354.20', 'utility: 292525')
 17: ('Name: 40, 5

## Hiện thực retail

In [ ]:
# chạy với retail
algorithm = UHUOPM()
algorithm.run_UHUOPM("./input_retail.txt, ./retail_utility.txt", 100)
algorithm.print_stats("../out/UHUOPM/retail/output_retail_top_100.txt")

Reading data . . .
 1: ('Name: 40, 49', 'prob: 7297.13', 'utility: 476144')
 2: ('Name: 49', 'prob: 21135.04', 'utility: 461182')
 3: ('Name: 40', 'prob: 25268.80', 'utility: 277044')
 4: ('Name: 33', 'prob: 7519.53', 'utility: 250500')
 5: ('Name: 40, 42', 'prob: 2824.13', 'utility: 247864')
 6: ('Name: 42, 49', 'prob: 2261.65', 'utility: 245579')
 7: ('Name: 42', 'prob: 7426.48', 'utility: 245544')
 8: ('Name: 40, 42, 49', 'prob: 910.21', 'utility: 238995')
 9: ('Name: 33, 49', 'prob: 1998.37', 'utility: 219766')
 10: ('Name: 33, 40', 'prob: 2084.56', 'utility: 184488')
 11: ('Name: 33, 40, 49', 'prob: 670.46', 'utility: 176116')
 12: ('Name: 39, 49', 'prob: 1978.98', 'utility: 173210')
 13: ('Name: 39', 'prob: 7775.43', 'utility: 169166')
 14: ('Name: 39, 40', 'prob: 2602.54', 'utility: 168244')
 15: ('Name: 39, 40, 49', 'prob: 763.77', 'utility: 165797')
 16: ('Name: 37, 39', 'prob: 689.54', 'utility: 150880')
 17: ('Name: 37', 'prob: 1470.53', 'utility: 128112')
 18: ('Name: 39, 4

In [ ]:
# chạy với retail
algorithm = UHUOPM()
algorithm.run_UHUOPM("./input_retail.txt, ./retail_utility.txt", 300)
algorithm.print_stats("../out/UHUOPM/retail/output_retail_top_300.txt")

Reading data . . .
 1: ('Name: 40, 49', 'prob: 7297.13', 'utility: 476144')
 2: ('Name: 49', 'prob: 21135.04', 'utility: 461182')
 3: ('Name: 40', 'prob: 25268.80', 'utility: 277044')
 4: ('Name: 33', 'prob: 7519.53', 'utility: 250500')
 5: ('Name: 40, 42', 'prob: 2824.13', 'utility: 247864')
 6: ('Name: 42, 49', 'prob: 2261.65', 'utility: 245579')
 7: ('Name: 42', 'prob: 7426.48', 'utility: 245544')
 8: ('Name: 40, 42, 49', 'prob: 910.21', 'utility: 238995')
 9: ('Name: 33, 49', 'prob: 1998.37', 'utility: 219766')
 10: ('Name: 33, 40', 'prob: 2084.56', 'utility: 184488')
 11: ('Name: 33, 40, 49', 'prob: 670.46', 'utility: 176116')
 12: ('Name: 39, 49', 'prob: 1978.98', 'utility: 173210')
 13: ('Name: 39', 'prob: 7775.43', 'utility: 169166')
 14: ('Name: 39, 40', 'prob: 2602.54', 'utility: 168244')
 15: ('Name: 39, 40, 49', 'prob: 763.77', 'utility: 165797')
 16: ('Name: 37, 39', 'prob: 689.54', 'utility: 150880')
 17: ('Name: 37', 'prob: 1470.53', 'utility: 128112')
 18: ('Name: 37, 3

In [ ]:
# chạy với retail
algorithm = UHUOPM()
algorithm.run_UHUOPM("./input_retail.txt, ./retail_utility.txt", 500)
algorithm.print_stats("../out/UHUOPM/retail/output_retail_top_500.txt")

Reading data . . .
 1: ('Name: 40, 49', 'prob: 7297.13', 'utility: 476144')
 2: ('Name: 49', 'prob: 21135.04', 'utility: 461182')
 3: ('Name: 40', 'prob: 25268.80', 'utility: 277044')
 4: ('Name: 33', 'prob: 7519.53', 'utility: 250500')
 5: ('Name: 40, 42', 'prob: 2824.13', 'utility: 247864')
 6: ('Name: 42, 49', 'prob: 2261.65', 'utility: 245579')
 7: ('Name: 42', 'prob: 7426.48', 'utility: 245544')
 8: ('Name: 40, 42, 49', 'prob: 910.21', 'utility: 238995')
 9: ('Name: 33, 49', 'prob: 1998.37', 'utility: 219766')
 10: ('Name: 33, 40', 'prob: 2084.56', 'utility: 184488')
 11: ('Name: 33, 40, 49', 'prob: 670.46', 'utility: 176116')
 12: ('Name: 39, 49', 'prob: 1978.98', 'utility: 173210')
 13: ('Name: 39', 'prob: 7775.43', 'utility: 169166')
 14: ('Name: 39, 40', 'prob: 2602.54', 'utility: 168244')
 15: ('Name: 39, 40, 49', 'prob: 763.77', 'utility: 165797')
 16: ('Name: 37, 39', 'prob: 689.54', 'utility: 150880')
 17: ('Name: 37', 'prob: 1470.53', 'utility: 128112')
 18: ('Name: 37, 3

In [ ]:
# chạy với retail
algorithm = UHUOPM()
algorithm.run_UHUOPM("./input_retail.txt, ./retail_utility.txt", 700)
algorithm.print_stats("../out/UHUOPM/retail/output_retail_top_700.txt")

Reading data . . .
 1: ('Name: 40, 49', 'prob: 7297.13', 'utility: 476144')
 2: ('Name: 49', 'prob: 21135.04', 'utility: 461182')
 3: ('Name: 40', 'prob: 25268.80', 'utility: 277044')
 4: ('Name: 33', 'prob: 7519.53', 'utility: 250500')
 5: ('Name: 40, 42', 'prob: 2824.13', 'utility: 247864')
 6: ('Name: 42, 49', 'prob: 2261.65', 'utility: 245579')
 7: ('Name: 42', 'prob: 7426.48', 'utility: 245544')
 8: ('Name: 40, 42, 49', 'prob: 910.21', 'utility: 238995')
 9: ('Name: 33, 49', 'prob: 1998.37', 'utility: 219766')
 10: ('Name: 33, 40', 'prob: 2084.56', 'utility: 184488')
 11: ('Name: 33, 40, 49', 'prob: 670.46', 'utility: 176116')
 12: ('Name: 39, 49', 'prob: 1978.98', 'utility: 173210')
 13: ('Name: 39', 'prob: 7775.43', 'utility: 169166')
 14: ('Name: 39, 40', 'prob: 2602.54', 'utility: 168244')
 15: ('Name: 39, 40, 49', 'prob: 763.77', 'utility: 165797')
 16: ('Name: 37, 39', 'prob: 689.54', 'utility: 150880')
 17: ('Name: 37', 'prob: 1470.53', 'utility: 128112')
 18: ('Name: 37, 3

In [ ]:
# chạy với retail
algorithm = UHUOPM()
algorithm.run_UHUOPM("./input_retail.txt, ./retail_utility.txt", 900)
algorithm.print_stats("../out/UHUOPM/retail/output_retail_top_900.txt")

Reading data . . .
 1: ('Name: 40, 49', 'prob: 7297.13', 'utility: 476144')
 2: ('Name: 49', 'prob: 21135.04', 'utility: 461182')
 3: ('Name: 40', 'prob: 25268.80', 'utility: 277044')
 4: ('Name: 33', 'prob: 7519.53', 'utility: 250500')
 5: ('Name: 40, 42', 'prob: 2824.13', 'utility: 247864')
 6: ('Name: 42, 49', 'prob: 2261.65', 'utility: 245579')
 7: ('Name: 42', 'prob: 7426.48', 'utility: 245544')
 8: ('Name: 40, 42, 49', 'prob: 910.21', 'utility: 238995')
 9: ('Name: 33, 49', 'prob: 1998.37', 'utility: 219766')
 10: ('Name: 33, 40', 'prob: 2084.56', 'utility: 184488')
 11: ('Name: 33, 40, 49', 'prob: 670.46', 'utility: 176116')
 12: ('Name: 39, 49', 'prob: 1978.98', 'utility: 173210')
 13: ('Name: 39', 'prob: 7775.43', 'utility: 169166')
 14: ('Name: 39, 40', 'prob: 2602.54', 'utility: 168244')
 15: ('Name: 39, 40, 49', 'prob: 763.77', 'utility: 165797')
 16: ('Name: 37, 39', 'prob: 689.54', 'utility: 150880')
 17: ('Name: 37', 'prob: 1470.53', 'utility: 128112')
 18: ('Name: 37, 3